# 🚢 Boat-Tracker - Application moderne de Tracking de Bateaux




## Guide de Lancement


Ce guide explique comment configurer et lancer l'application de surveillance maritime dans un environnement Google Colab.

**Prérequis :**

*   Un compte GitHub avec accès au dépôt privé `https://github.com/doxav/CollabToolBuilder.git`.
*   Un jeton GitHub (Classic) avec l'autorisation `repo` pour cloner le dépôt privé.
*   Clés API pour les services externes utilisés (AIStream, OpenWeather, OpenRouter pour les modèles LLM).

**Étapes de Lancement :**

1.  **Ouvrir le Terminal Colab :** Dans Google Colab, ouvrez le terminal intégré (généralement accessible via le menu "Exécution" ou dans la barre latérale gauche si activé).

2.  **Cloner le Dépôt `CollabToolBuilder` :** Exécutez la commande suivante dans le terminal :

git clone https://github.com/doxav/CollabToolBuilder.git


Lorsque vous y êtes invité, entrez votre nom d'utilisateur GitHub et collez votre jeton (token) GitHub comme mot de passe. Ensuite, entrez ces commandes l'une apres l'autre :

    cd CollabToolBuilder

    pip install -e .


3.  **Insérer vos Clés API :** Avant d'exécuter les cellules du notebook, assurez-vous de renseigner vos clés API dans la cellule de configuration dédiée.

4.  **Exécuter Toutes les Cellules :** Cliquez sur "Exécution" -> "Tout exécuter" dans le menu de Colab. Laissez le notebook s'exécuter complètement.

5.  **Actualiser le Flux AIS Initial :** Une fois que toutes les cellules ont terminé de s'exécuter, attendez quelques instants pour permettre au flux AIS de collecter des données. Ensuite, **relancez la toute dernière cellule du notebook** (celle qui initie la connexion au flux AIS et met à jour la carte). Cela actualisera la liste des navires dans `vessels_state` et affichera le maximum de navires disponibles sur la carte.

6.  **Rafraîchir les Données AIS (Ultérieurement) :** Si à un moment donné vous souhaitez actualiser la position des navires et les données affichées sur la carte avec les informations les plus récentes du flux AIS, **il suffit de relancer la toute dernière cellule du notebook**.

Après avoir suivi ces étapes, l'interface interactive (carte, sélecteur de navires, chat) devrait être fonctionnelle et prête à être utilisée.

# 📦 Installation des dépendances Python



In [465]:
!pip install folium ipywidgets requests pandas plotly dash jupyter-dash geopy openai websockets nest_asyncio
!pip install -qU langchain langchain-openai langchain-community
!pip install copernicusmarine xarray netCDF4
!pip install shapely duckdb



📊 160200 messages AIS reçus, 1877 navires en mémoire


# 🔑 Configuration des clés API (OpenWeather, OpenRouter, AISStream)


In [466]:
OPENWEATHER_API_KEY = ""
OPENROUTER_API_KEY = ""
AISSTREAM_API_KEY = ""


# URLs des APIs
OPENWEATHER_BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1/chat/completions"

# Configuration alternative pour le chatbot local (si OpenRouter ne fonctionne pas)
USE_LOCAL_CHATBOT = False  # Mettre à True pour utiliser un chatbot local simple


# 🗂️ Imports et configuration de base

In [467]:
import pandas as pd
import folium
from folium import plugins
import requests
import json
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objects as go
import plotly.express as px
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import warnings
warnings.filterwarnings('ignore')

# Configuration pour l'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


#Historisation simple en mémoire + JSONL (sans BDD)

In [468]:
from collections import defaultdict, deque
import os, datetime as dt

WATCH_MMSI = "271051231"

# Fenêtre de rétention en mémoire (ring buffer) et archive journalière JSONL
HISTORY_WINDOW_MIN = 120
MAX_POINTS_PER_VESSEL = 5000
ais_history = defaultdict(lambda: deque(maxlen=MAX_POINTS_PER_VESSEL))

ARCHIVE_DIR = "/content/ais_archive"
os.makedirs(ARCHIVE_DIR, exist_ok=True)

def _append_history(mmsi, ts, lat, lon, sog, cog):
    """Alimente la mémoire courte + écrit en JSONL (une ligne par point)"""
    rec = {"mmsi": str(mmsi), "ts": ts, "lat": lat, "lon": lon, "sog": sog, "cog": cog}
    if str(mmsi) == str(WATCH_MMSI):
        print(f"🟢 append_history {mmsi} @ {ts} lat={lat:.5f} lon={lon:.5f} sog={sog} cog={cog}")

    ais_history[str(mmsi)].append(rec)
    day = dt.datetime.utcfromtimestamp(ts).strftime("%Y%m%d")
    with open(f"{ARCHIVE_DIR}/positions_{day}.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")


# 🧠 Configuration HumanLLM maritime


In [469]:
#respecifier les clées api pour la config de l'outil humanllm

OPENWEATHER_API_KEY = ""
OPENROUTER_API_KEY = ""
AISSTREAM_API_KEY = ""

print("✅ Clés API redéfinies")


from langchain_openai import ChatOpenAI

try:
    maritime_humanllm = HumanLLM(
        system_prompt="""Tu es un assistant naval opérationnel expert, spécialisé dans l'aide à la navigation maritime.

CONTEXTE OPÉRATIONNEL:
- Tu assistes des marins professionnels dans leurs opérations
- Tes réponses doivent être claires, précises et adaptées au vocabulaire maritime
- Utilise toujours les unités nautiques (nœuds, milles nautiques, etc.)

OUTILS OBLIGATOIRES:
- Pour toute question de position/localisation : utilise ais_lookup
- Pour l'évaluation de risque : utilise risk_assess

STYLE DE COMMUNICATION:
- Sois direct et factuel
- Structure tes réponses avec des puces claires
- Utilise des emojis pour la lisibilité (📍 🌊 ⚡ 🧭)
- Indique toujours le MMSI pour identification
- Signale les points d'attention avec ⚠️ ou 🚨""",

        llmORchains_list={
            "maritime_main": ChatOpenAI(
                model="deepseek/deepseek-chat-v3.1:free",
                temperature=0.2,
                api_key=OPENROUTER_API_KEY,
                base_url="https://openrouter.ai/api/v1",
                default_headers={
                    "HTTP-Referer": "https://colab.research.google.com",
                    "X-Title": "Maritime HumanLLM"
                }
            )
        },

        skip_rounds=5,  # Intervention de l'outil toutes les 5 requetes

        # Optionnel : configuration par défaut
        default_llmORchain="maritime_main"
    )

    print("✅ HumanLLM maritime configuré avec succès !")
    print(f"📊 Configuration: skip_rounds={maritime_humanllm.skip_rounds}")

except Exception as e:
    print(f"❌ Erreur configuration HumanLLM: {e}")
    print("🔍 Détails de l'erreur:")
    import traceback
    traceback.print_exc()

📊 152100 messages AIS reçus, 2107 navires en mémoire
📊 152200 messages AIS reçus, 2107 navires en mémoire
📊 152300 messages AIS reçus, 2107 navires en mémoire
📊 196700 messages AIS reçus, 2110 navires en mémoire
📊 196800 messages AIS reçus, 2115 navires en mémoire
📊 196900 messages AIS reçus, 2122 navires en mémoire
📊 197000 messages AIS reçus, 2127 navires en mémoire
✅ Clés API redéfinies
LLM/Chain 'premium_llm' not found in llmORchains_list ['maritime_main']
✅ HumanLLM maritime configuré avec succès !
📊 Configuration: skip_rounds=5


# 🏳️ Dictionnaire des drapeaux par pays

In [470]:
FLAG_EMOJIS = {
    # Pays des bateaux de la flotte
    'France': '🇫🇷',
    'Royaume-Uni': '🇬🇧',
    'États-Unis': '🇺🇸',
    'Japon': '🇯🇵',
    'Italie': '🇮🇹',
    'Suède': '🇸🇪',
    'Finlande': '🇫🇮',
    'Bahamas': '🇧🇸',
    'Russie': '🇷🇺',
    'Pays-Bas': '🇳🇱',
    'Hong Kong': '🇭🇰',
    'Chine': '🇨🇳',
    'Islande': '🇮🇸',
    'Belgique': '🇧🇪',
    # Pays supplémentaires pour la géolocalisation
    'Espagne': '🇪🇸',
    'Allemagne': '🇩🇪',
    'Portugal': '🇵🇹',
    'Norvège': '🇳🇴',
    'Danemark': '🇩🇰',
    'Grèce': '🇬🇷',
    'Turquie': '🇹🇷',
    'Maroc': '🇲🇦',
    'Algérie': '🇩🇿',
    'Tunisie': '🇹🇳',
    'Égypte': '🇪🇬',
    'Canada': '🇨🇦',
    'Mexique': '🇲🇽',
    'Brésil': '🇧🇷',
    'Argentine': '🇦🇷',
    'Chili': '🇨🇱',
    'Australie': '🇦🇺',
    'Nouvelle-Zélande': '🇳🇿',
    'Afrique du Sud': '🇿🇦',
    'Inde': '🇮🇳',
    'Corée du Sud': '🇰🇷',
    'Thaïlande': '🇹🇭',
    'Indonésie': '🇮🇩',
    'Philippines': '🇵🇭',
    'Singapour': '🇸🇬',
    'Malaisie': '🇲🇾',
    'Vietnam': '🇻🇳',
    'Israël': '🇮🇱',
    'Émirats arabes unis': '🇦🇪',
    'Arabie saoudite': '🇸🇦',

    # Noms anglais pour compatibilité avec l'API de géolocalisation
    'United Kingdom': '🇬🇧',
    'United States': '🇺🇸',
    'Japan': '🇯🇵',
    'Italy': '🇮🇹',
    'Sweden': '🇸🇪',
    'Finland': '🇫🇮',
    'Russia': '🇷🇺',
    'Netherlands': '🇳🇱',
    'China': '🇨🇳',
    'Iceland': '🇮🇸',
    'Belgium': '🇧🇪',
    'Spain': '🇪🇸',
    'Germany': '🇩🇪',
    'Norway': '🇳🇴',
    'Denmark': '🇩🇰',
    'Greece': '🇬🇷',
    'Turkey': '🇹🇷',
    'Morocco': '🇲🇦',
    'Algeria': '🇩🇿',
    'Tunisia': '🇹🇳',
    'Egypt': '🇪🇬',
    'Mexico': '🇲🇽',
    'Brazil': '🇧🇷',
    'Argentina': '🇦🇷',
    'Chile': '🇨🇱',
    'Australia': '🇦🇺',
    'New Zealand': '🇳🇿',
    'South Africa': '🇿🇦',
    'India': '🇮🇳',
    'South Korea': '🇰🇷',
    'Thailand': '🇹🇭',
    'Indonesia': '🇮🇩',
    'Philippines': '🇵🇭',
    'Singapore': '🇸🇬',
    'Malaysia': '🇲🇾',
    'Vietnam': '🇻🇳',
    'Israel': '🇮🇱',
    'United Arab Emirates': '🇦🇪',
    'Saudi Arabia': '🇸🇦',
    # Cas spéciaux
    'En mer': '🌊',
    'Inconnu': '🏳️',
    'Erreur': '❌'
}

def get_flag_emoji(country):
    """Retourne l'emoji du drapeau pour un pays donné"""
    return FLAG_EMOJIS.get(country, '🏳️')  # Drapeau blanc par défaut

# 🌤️ Fonctions API météo OpenWeather

In [471]:
def get_weather_data(lat, lon, api_key):
    """
    Récupère les données météo pour une position donnée
    """
    try:
        url = f"{OPENWEATHER_BASE_URL}?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=fr"
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Erreur API: {response.status_code}"}
    except Exception as e:
        return {"error": f"Erreur de connexion: {str(e)}"}

def format_weather_info(weather_data):
    """
    Formate les informations météo pour l'affichage
    """
    if "error" in weather_data:
        return f"❌ {weather_data['error']}"

    try:
        temp = weather_data['main']['temp']
        feels_like = weather_data['main']['feels_like']
        humidity = weather_data['main']['humidity']
        pressure = weather_data['main']['pressure']
        description = weather_data['weather'][0]['description'].title()
        wind_speed = weather_data['wind']['speed']
        wind_deg = weather_data['wind'].get('deg', 0)

        weather_info = f"""
        🌡️ **Température**: {temp}°C (ressenti {feels_like}°C)
        🌤️ **Conditions**: {description}
        💨 **Vent**: {wind_speed} m/s, {wind_deg}°
        💧 **Humidité**: {humidity}%
        📊 **Pression**: {pressure} hPa
        """

        return weather_info.strip()
    except KeyError as e:
        return f"❌ Données météo incomplètes: {str(e)}"

print("✅ Fonctions météo configurées!")



✅ Fonctions météo configurées!


# 🌍 Fonctions de géolocalisation et zones maritimes

In [472]:
# 🌍 Fonctions de géolocalisation avec zones maritimes précises (eaux territoriales)

import math

def get_location_info(lat, lon):
    """
    Récupère les informations de localisation avec zones maritimes précises
    """
    try:
        geolocator = Nominatim(user_agent="boat_tracker_app")
        location = geolocator.reverse(
            f"{lat}, {lon}",
            timeout=10,
            language='fr'
        )

        if location:
            # Cas 1: Position terrestre/portuaire détectée par OSM
            address = location.raw['address']
            country = translate_to_french(address.get('country', 'Inconnu'))
            city = translate_to_french(
                address.get('city') or
                address.get('town') or
                address.get('village') or
                'Zone portuaire'
            )
            state = translate_to_french(address.get('state', ''))
            water_body = get_water_body(lat, lon)

            return {
                'country': country,
                'city': city,
                'state': state,
                'water_body': water_body,
                'full_address': translate_to_french(location.address),
                'maritime_status': 'Port/Côte'
            }
        else:
            # Cas 2: Position maritime - analyser les zones
            maritime_zone = get_precise_maritime_zone(lat, lon)

            return {
                'country': maritime_zone['country'],
                'city': maritime_zone['zone_type'],
                'state': '',
                'water_body': maritime_zone['water_body'],
                'full_address': maritime_zone['description'],
                'maritime_status': maritime_zone['status']
            }

    except GeocoderTimedOut:
        return {
            'country': 'Timeout',
            'city': 'Erreur de géolocalisation',
            'state': '',
            'water_body': 'Inconnu',
            'full_address': 'Erreur de connexion',
            'maritime_status': 'Erreur'
        }
    except Exception as e:
        return {
            'country': 'Erreur',
            'city': str(e),
            'state': '',
            'water_body': 'Inconnu',
            'full_address': f"Erreur: {str(e)}",
            'maritime_status': 'Erreur'
        }

def get_precise_maritime_zone(lat, lon):
    """
    Détermine la zone maritime précise selon le droit international
    """
    # 1. Déterminer la masse d'eau
    water_body = get_water_body(lat, lon)

    # 2. Trouver le pays côtier le plus proche
    closest_country_info = find_closest_coastal_country(lat, lon)
    closest_country = closest_country_info['country']
    distance_nm = closest_country_info['distance_nm']

    # 3. Déterminer la zone maritime selon le droit international
    if distance_nm <= 12:  # Eaux territoriales (12 milles nautiques)
        zone_type = "Eaux territoriales"
        country = closest_country
        status = f"Territorial ({distance_nm:.1f} nm de la côte)"
        description = f"Eaux territoriales {closest_country} - {distance_nm:.1f} nm de la côte"

    elif distance_nm <= 24:  # Zone contiguë (24 nm)
        zone_type = "Zone contiguë"
        country = f"Zone contiguë {closest_country}"
        status = f"Contiguë ({distance_nm:.1f} nm de la côte)"
        description = f"Zone contiguë {closest_country} - {distance_nm:.1f} nm de la côte"

    elif distance_nm <= 200:  # Zone économique exclusive (200 nm)
        zone_type = "ZEE"
        country = f"ZEE {closest_country}"
        status = f"ZEE ({distance_nm:.1f} nm de la côte)"
        description = f"Zone Économique Exclusive {closest_country} - {distance_nm:.1f} nm de la côte"

    else:  # Eaux internationales (> 200 nm)
        zone_type = "Eaux internationales"
        country = "Eaux internationales"
        status = f"International ({distance_nm:.1f} nm de la côte)"
        description = f"Eaux internationales - {distance_nm:.1f} nm de {closest_country}"

    return {
        'country': country,
        'zone_type': zone_type,
        'water_body': water_body,
        'status': status,
        'description': description,
        'distance_nm': distance_nm,
        'closest_country': closest_country
    }

def find_closest_coastal_country(lat, lon):
    """
    Trouve le pays côtier le plus proche et calcule la distance
    """
    # Points côtiers de référence pour les principaux pays méditerranéens
    coastal_references = {
        'France': [
            (43.2965, 5.3698),   # Marseille
            (43.7102, 7.2620),   # Nice
            (42.6976, 2.8954),   # Perpignan/Port-Vendres
            (43.1258, 6.1266),   # Toulon
            (43.5528, 3.8330),   # Montpellier/Sète
        ],
        'Espagne': [
            (41.3851, 2.1734),   # Barcelone
            (36.7201, -4.4203),  # Malaga
            (37.3886, -5.9823),  # Séville
            (39.4699, 2.7581),   # Palma de Majorque
            (36.1408, -5.3536),  # Gibraltar
        ],
        'Italie': [
            (40.8518, 14.2681),  # Naples
            (41.9028, 12.4964),  # Rome/Civitavecchia
            (44.4056, 8.9463),   # Gênes
            (45.4408, 12.3155),  # Venise
            (38.1157, 13.3615),  # Palerme
            (40.6401, 17.9443),  # Brindisi
        ],
        'Grèce': [
            (37.9755, 23.7348),  # Athènes/Pirée
            (40.6401, 22.9444),  # Thessalonique
            (35.5138, 24.0180),  # Crète/Héraklion
            (39.6243, 19.9217),  # Corfou
            (36.4414, 25.3656),  # Santorin
        ],
        'Turquie': [
            (41.0082, 28.9784),  # Istanbul
            (36.8969, 30.7133),  # Antalya
            (38.4192, 27.1287),  # Izmir
            (36.2012, 36.1610),  # Antakya
        ],
        'Chypre': [
            (35.1856, 33.3823),  # Nicosie/Limassol
            (35.0408, 33.9533),  # Larnaca
        ],
        'Croatie': [
            (45.8150, 15.9819),  # Zagreb/Rijeka
            (42.6507, 18.0944),  # Dubrovnik
            (43.5081, 16.4402),  # Split
        ],
        'Algérie': [
            (36.7538, 3.0588),   # Alger
            (35.6976, -0.6337),  # Oran
        ],
        'Tunisie': [
            (36.8065, 10.1815),  # Tunis
            (34.7406, 10.7603),  # Sfax
        ],
        'Maroc': [
            (35.7595, -5.8340),  # Tanger
            (33.5731, -7.5898),  # Casablanca
        ],
        'Égypte': [
            (31.2001, 29.9187),  # Alexandrie
            (27.2579, 33.8116),  # Hurghada
        ],
        'Malte': [
            (35.8997, 14.5146),  # La Valette
        ],
        'Monaco': [
            (43.7384, 7.4246),   # Monaco
        ]
    }

    min_distance = float('inf')
    closest_country = 'Inconnu'

    for country, points in coastal_references.items():
        for point_lat, point_lon in points:
            distance = haversine_nm(lat, lon, point_lat, point_lon)
            if distance < min_distance:
                min_distance = distance
                closest_country = country

    return {
        'country': closest_country,
        'distance_nm': min_distance
    }

def haversine_nm(lat1, lon1, lat2, lon2):
    """
    Calcule la distance en milles nautiques entre deux points GPS
    """
    R = 3440.065  # Rayon de la Terre en milles nautiques

    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    return R * c

def get_water_body(lat, lon):
    """
    Détermine la masse d'eau avec plus de précision
    """
    # Méditerranée (étendue)
    if 30 <= lat <= 46 and -6 <= lon <= 42:
        # Sous-zones méditerranéennes
        if 40 <= lat <= 46 and 3 <= lon <= 20:
            return "Mer Méditerranée occidentale"
        elif 32 <= lat <= 42 and 15 <= lon <= 30:
            return "Mer Méditerranée orientale"
        elif 35 <= lat <= 42 and 12 <= lon <= 20:
            return "Mer Adriatique"
        elif 35 <= lat <= 40 and 19 <= lon <= 30:
            return "Mer Ionienne"
        elif 35 <= lat <= 42 and 22 <= lon <= 30:
            return "Mer Égée"
        else:
            return "Mer Méditerranée"

    # Mer Noire
    elif 40 <= lat <= 48 and 27 <= lon <= 42:
        return "Mer Noire"

    # Mer du Nord / Baltique
    elif 53 <= lat <= 70 and -5 <= lon <= 30:
        if lat >= 60:
            return "Mer de Norvège"
        elif 54 <= lat <= 66 and 10 <= lon <= 30:
            return "Mer Baltique"
        else:
            return "Mer du Nord"

    # Océan Atlantique
    elif -60 <= lat <= 70 and -80 <= lon <= 0:
        if 35 <= lat <= 70:
            return "Océan Atlantique Nord"
        elif 0 <= lat <= 35:
            return "Océan Atlantique tropical"
        else:
            return "Océan Atlantique Sud"

    # Océan Pacifique
    elif -60 <= lat <= 60 and 120 <= lon <= 180:
        return "Océan Pacifique"

    # Océan Indien
    elif -60 <= lat <= 30 and 20 <= lon <= 120:
        return "Océan Indien"

    # Océan Arctique
    elif lat >= 70:
        return "Océan Arctique"

    # Océan Antarctique
    elif lat <= -60:
        return "Océan Antarctique"

    else:
        return "Zone maritime"

def translate_to_french(text):
    """
    Traduit les noms de lieux vers le français avec dictionnaire de correspondances
    """
    if not text or text in ['', 'Inconnu', 'En mer', 'Erreur']:
        return text

    # Dictionnaire de traductions (identique à avant mais étendu)
    translations = {
        # Grèce
        'Ελλάς': 'Grèce', 'Ελλάδα': 'Grèce', 'Greece': 'Grèce',
        'Κυψέλη': 'Kypseli', 'Αθήνα': 'Athènes', 'Θεσσαλονίκη': 'Thessalonique',
        'Πειραιάς': 'Le Pirée', 'Πάτρα': 'Patras', 'Κρήτη': 'Crète',
        'Σαντορίνη': 'Santorin', 'Μύκονος': 'Mykonos', 'Ρόδος': 'Rhodes',
        'Κέρκυρα': 'Corfou', 'Επισκοπή': 'Episkopi', 'Επίσκοπι': 'Episkopi',

        # Chypre
        'Cyprus': 'Chypre', 'Κύπρος': 'Chypre', 'Kıbrıs': 'Chypre',
        'Episkopi': 'Episkopi', 'Επισκοπή': 'Episkopi',

        # Espagne
        'España': 'Espagne', 'Barcelona': 'Barcelone', 'Sevilla': 'Séville',
        'Málaga': 'Malaga', 'Valencia': 'Valence', 'Zaragoza': 'Saragosse',

        # Italie
        'Italia': 'Italie', 'Roma': 'Rome', 'Milano': 'Milan',
        'Napoli': 'Naples', 'Venezia': 'Venise', 'Firenze': 'Florence',
        'Genova': 'Gênes', 'Palermo': 'Palerme',

        # Turquie
        'Türkiye': 'Turquie', 'Turkey': 'Turquie', 'İstanbul': 'Istanbul',
        'İzmir': 'Izmir', 'Antalya': 'Antalya',

        # Pays arabes
        'المغرب': 'Maroc', 'Morocco': 'Maroc', 'مصر': 'Égypte', 'Egypt': 'Égypte',
        'الجزائر': 'Algérie', 'Algeria': 'Algérie', 'تونس': 'Tunisie', 'Tunisia': 'Tunisie',

        # Autres
        'Malta': 'Malte', 'Croatia': 'Croatie', 'Hrvatska': 'Croatie',
        'Albania': 'Albanie', 'Shqipëria': 'Albanie',
        'United States': 'États-Unis', 'United Kingdom': 'Royaume-Uni',
        'Germany': 'Allemagne', 'Netherlands': 'Pays-Bas',
    }

    # Recherche exacte puis partielle
    if text in translations:
        return translations[text]

    result = text
    for original, french in translations.items():
        if original in result:
            result = result.replace(original, french)

    return result

def get_country_flag_smart(country_name):
    """Détecte intelligemment le drapeau d'un pays même avec des noms multilingues"""
    if not country_name or country_name in ['Inconnu', 'En mer', 'Erreur']:
        return get_flag_emoji(country_name)

    country_lower = country_name.lower()

    # Mappings étendus pour les zones maritimes
    country_mappings = {
        'france': '🇫🇷', 'grèce': '🇬🇷', 'greece': '🇬🇷', 'ελλάς': '🇬🇷', 'ελλάδα': '🇬🇷',
        'espagne': '🇪🇸', 'spain': '🇪🇸', 'españa': '🇪🇸',
        'italie': '🇮🇹', 'italy': '🇮🇹', 'italia': '🇮🇹',
        'turquie': '🇹🇷', 'turkey': '🇹🇷', 'türkiye': '🇹🇷',
        'chypre': '🇨🇾', 'cyprus': '🇨🇾', 'κύπρος': '🇨🇾',
        'croatie': '🇭🇷', 'croatia': '🇭🇷', 'hrvatska': '🇭🇷',
        'algérie': '🇩🇿', 'algeria': '🇩🇿', 'الجزائر': '🇩🇿',
        'tunisie': '🇹🇳', 'tunisia': '🇹🇳', 'تونس': '🇹🇳',
        'maroc': '🇲🇦', 'morocco': '🇲🇦', 'المغرب': '🇲🇦',
        'égypte': '🇪🇬', 'egypt': '🇪🇬', 'مصر': '🇪🇬',
        'malte': '🇲🇹', 'malta': '🇲🇹',
        'albanie': '🇦🇱', 'albania': '🇦🇱',
        'monténégro': '🇲🇪', 'montenegro': '🇲🇪',
        'eaux internationales': '🌊', 'international': '🌊',
        'zee': '🌊', 'zone contiguë': '🌊', 'eaux territoriales': '🌊',
    }

    # Recherche avec gestion des zones maritimes
    for key, flag in country_mappings.items():
        if key in country_lower:
            return flag

    return get_flag_emoji(country_name)

def format_location_info(location_data):
    """Formate les informations de localisation avec zones maritimes précises"""
    country = location_data['country']
    flag = get_country_flag_smart(country)

    # Affichage adapté selon le type de zone
    if location_data.get('maritime_status') == 'Port/Côte':
        location_info = f"""
        {flag} **Pays**: {country}
        🏙️ **Ville**: {location_data['city']}
        🌊 **Zone maritime**: {location_data['water_body']}
        📍 **Adresse complète**: {location_data['full_address']}
        """
    else:
        # Zone maritime
        status = location_data.get('maritime_status', '')
        location_info = f"""
        {flag} **Zone**: {country}
        🌊 **Statut**: {location_data['city']} ({status})
        🌊 **Masse d'eau**: {location_data['water_body']}
        📍 **Position**: {location_data['full_address']}
        """

    return location_info.strip()

print("✅ Fonctions de géolocalisation configurées avec zones maritimes précises!")

✅ Fonctions de géolocalisation configurées avec zones maritimes précises!


In [473]:
def on_boat_selection_change(change):
    """Gère le changement de sélection de navire"""
    global last_selected_mmsi

    selected_mmsi = change['new']
    last_selected_mmsi = selected_mmsi

    if selected_mmsi is None:
        # Aucun navire sélectionné
        weather_output.value = "<div style='padding:10px;background:#f8f9fa;border-radius:5px;color:#6c757d;'>🌤️ Sélectionnez un navire pour voir la météo</div>"
        location_output.value = "<div style='padding:10px;background:#f8f9fa;border-radius:5px;color:#6c757d;'>🌍 Sélectionnez un navire pour voir la géolocalisation</div>"
        return

    try:
        # Récupérer les données du navire sélectionné depuis vessels_state
        vessel_data = vessels_state.get(str(selected_mmsi))

        if not vessel_data:
            # Si pas dans vessels_state, essayer depuis le DataFrame
            df = vessels_state_to_df()
            vessel_row = df[df['mmsi'] == selected_mmsi]

            if vessel_row.empty:
                weather_output.value = "<div style='padding:10px;background:#fff3cd;border-radius:5px;color:#856404;'>⚠️ Navire non trouvé dans les données AIS</div>"
                location_output.value = "<div style='padding:10px;background:#fff3cd;border-radius:5px;color:#856404;'>⚠️ Navire non trouvé dans les données AIS</div>"
                return

            # Récupérer les coordonnées depuis le DataFrame
            lat = float(vessel_row.iloc[0]['latitude'])
            lon = float(vessel_row.iloc[0]['longitude'])
            vessel_name = vessel_row.iloc[0]['boat_name']
        else:
            # Récupérer les coordonnées depuis vessels_state
            lat = float(vessel_data.get("lat", 0))
            lon = float(vessel_data.get("lon", 0))
            vessel_name = vessel_data.get("name", f"MMSI_{selected_mmsi}")

        # Vérifier que les coordonnées sont valides
        if lat == 0 and lon == 0:
            weather_output.value = "<div style='padding:10px;background:#f8d7da;border-radius:5px;color:#721c24;'>❌ Coordonnées invalides pour ce navire</div>"
            location_output.value = "<div style='padding:10px;background:#f8d7da;border-radius:5px;color:#721c24;'>❌ Coordonnées invalides pour ce navire</div>"
            return

        print(f" Navire sélectionné: {vessel_name} (MMSI: {selected_mmsi})")
        print(f" Coordonnées: {lat:.4f}°N, {lon:.4f}°E")



    except Exception as e:
        print(f"❌ Erreur sélection navire: {e}")
        weather_output.value = f"<div style='padding:10px;background:#f8d7da;border-radius:5px;color:#721c24;'>❌ Erreur: {e}</div>"
        location_output.value = f"<div style='padding:10px;background:#f8d7da;border-radius:5px;color:#721c24;'>❌ Erreur: {e}</div>"

print("✅ Fonction on_boat_selection_change créée")

✅ Fonction on_boat_selection_change créée


# 🤖 Fonctions chatbot (OpenRouter + fallback local)

In [474]:
def query_openrouter(question, context_data, api_key):
    """
    Interroge l'API OpenRouter avec les données contextuelles
    """
    try:
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://colab.research.google.com",
            "X-Title": "Boat Tracker App"
        }

        # Préparation du contexte
        context = f"""
        Tu es un assistant spécialisé dans le tracking maritime.
        Voici les données des bateaux disponibles:
        {context_data}

        Réponds de manière précise et utile aux questions sur ces bateaux.
        """

        payload = {
            "model": "deepseek/deepseek-chat-v3.1:free",
            "messages": [
                {"role": "system", "content": context},
                {"role": "user", "content": question}
            ],
            "max_tokens": 500,
            "temperature": 0.7
        }

        response = requests.post(OPENROUTER_BASE_URL,
                               headers=headers,
                               json=payload,
                               timeout=30)

        if response.status_code == 200:
            result = response.json()
            return result['choices'][0]['message']['content']
        elif response.status_code == 402:
            return "❌ Erreur 402: Crédit insuffisant sur votre compte OpenRouter. Vérifiez votre solde sur https://openrouter.ai/credits"
        elif response.status_code == 401:
            return "❌ Erreur 401: Clé API OpenRouter invalide. Vérifiez votre clé sur https://openrouter.ai/keys"
        elif response.status_code == 429:
            return "❌ Erreur 429: Limite de taux dépassée. Attendez quelques minutes avant de réessayer."
        else:
            return f"❌ Erreur API OpenRouter: {response.status_code} - {response.text}"

    except Exception as e:
        return f"❌ Erreur de connexion au chatbot: {str(e)}"

def local_chatbot(question, context_data):
    """
    Chatbot local simple basé sur des règles (alternative à OpenRouter)
    """
    question_lower = question.lower()

    # Comptage des bateaux
    if any(word in question_lower for word in ['combien', 'nombre', 'total']):
        boat_count = context_data.count('⚪') + context_data.count('🔴')
        return f"Il y a {boat_count} bateaux dans la flotte actuellement trackés."

    # Types de bateaux
    if any(word in question_lower for word in ['type', 'catégorie', 'sorte']):
        types = ['Cargo', 'Research', 'Fishing', 'Container', 'Cruise', 'Tanker', 'Naval', 'Icebreaker', 'Tourism', 'Yacht']
        return f"Les types de bateaux dans la flotte sont : {', '.join(types)}"

    # Pays/pavillons
    if any(word in question_lower for word in ['pays', 'pavillon', 'nationalité', 'drapeau']):
        countries = ['France', 'United Kingdom', 'United States', 'Japan', 'Italy', 'Sweden', 'Finland', 'Bahamas', 'Russia', 'Netherlands', 'Hong Kong', 'China', 'Iceland', 'Belgium']
        return f"Les pavillons représentés sont : {', '.join(set(countries))}"

    # Bateau sélectionné
    if '🔴' in context_data:
        selected_line = [line for line in context_data.split('\n') if '🔴' in line]
        if selected_line:
            boat_info = selected_line[0].replace('🔴', '').strip()
            if any(word in question_lower for word in ['sélectionné', 'choisi', 'actuel', 'rouge']):
                return f"Le bateau actuellement sélectionné est : {boat_info}"

    # Localisation
    if any(word in question_lower for word in ['où', 'position', 'localisation', 'coordonnées']):
        return "Les bateaux sont répartis dans le monde entier. Sélectionnez un bateau pour voir sa position exacte et les informations de localisation."

    # Météo
    if any(word in question_lower for word in ['météo', 'temps', 'température', 'vent']):
        return "Les informations météo sont disponibles pour chaque bateau sélectionné. Choisissez un bateau pour voir les conditions météorologiques en temps réel."

    # Réponse par défaut
    return f"Je suis un assistant maritime local. Voici ce que je peux vous dire :\n\n{context_data}\n\nPosez-moi des questions sur le nombre de bateaux, leurs types, leurs pavillons, ou sélectionnez un bateau pour plus d'informations."

def create_context_from_boats(df, selected_boat=None):
    """
    Crée un contexte textuel à partir des données des bateaux
    """
    context = "Données des bateaux:\n"

    for _, boat in df.iterrows():
        marker = "🔴" if selected_boat and boat['boat_name'] == selected_boat else "⚪"
        context += f"{marker} {boat['boat_name']}: {boat['boat_type']}, {boat['flag_country']}, Position: ({boat['latitude']:.4f}, {boat['longitude']:.4f})\n"

    return context

print("✅ Fonctions chatbot configurées!")


✅ Fonctions chatbot configurées!


# 🗺️ Fonction création carte Folium avec bathymétrie EMODnet

In [475]:
# === Helper pour détecter automatiquement les noms de layers WMS EMODnet ===
import xml.etree.ElementTree as ET
import requests

def wms_find_layer_name(service_url: str, keywords=("cable",), timeout=10):
    """
    Télécharge le GetCapabilities et renvoie le premier layer dont le Title/Name
    contient TOUS les keywords (sans casse). Retourne None si introuvable.
    """
    try:
        caps = requests.get(
            service_url,
            params={"SERVICE":"WMS","REQUEST":"GetCapabilities","VERSION":"1.3.0"},
            timeout=timeout
        )
        caps.raise_for_status()
        root = ET.fromstring(caps.content)
        ns = {"wms":"http://www.opengis.net/wms"}
        # compatible avec diverses arborescences, on scanne tous les 'Layer'
        for ly in root.findall(".//wms:Layer", ns):
            title = "".join((ly.findtext("wms:Title", default="", namespaces=ns) or "")).lower()
            name  = "".join((ly.findtext("wms:Name",  default="", namespaces=ns) or "")).lower()
            hay = title + " " + name
            if all(k.lower() in hay for k in keywords):
                nm = ly.findtext("wms:Name", default="", namespaces=ns)
                if nm:
                    return nm
    except Exception:
        return None
    return None

def create_boat_map(df, selected_mmsi=None):
    """Carte maritime simple et propre """
    center_lat, center_lon = 38.5, 15.0
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=5,
        tiles='CartoDB dark_matter',
        prefer_canvas=True
    )

    # Couche bathymétrie EMODnet
    folium.WmsTileLayer(
        url="https://ows.emodnet-bathymetry.eu/wms",
        name="Bathymétrie EMODnet",
        layers="emodnet:mean_atlas_land",
        fmt="image/png",
        transparent=True,
        overlay=True,
        control=True,
        attr="EMODnet Bathymetry",
        opacity=0.6
    ).add_to(m)

     # 🌐 WMS EMODnet Human Activities - on tente de trouver les layers automatiquement
    HA_WMS = "https://ows.emodnet-humanactivities.eu/wms"

    try:
        # 1) Câbles sous-marins
        cable_layer = wms_find_layer_name(HA_WMS, keywords=("cable",))
        if cable_layer:
            folium.WmsTileLayer(
                url=HA_WMS,
                name="Câbles sous-marins (EMODnet)",
                layers=cable_layer,     # ex (selon millésime): emodnet:submarine_cables ou similaire
                fmt="image/png",
                transparent=True, overlay=True, control=True, opacity=0.7,
                attr="EMODnet Human Activities"
            ).add_to(m)

        # 2) Densité des routes/navires
        #    Mots-clés possibles: “vessel”, “density”, “ship”. On essaye “vessel” + “density”.
        density_layer = wms_find_layer_name(HA_WMS, keywords=("vessel","density"))
        if density_layer:
            folium.WmsTileLayer(
                url=HA_WMS,
                name="Densité de trafic (EMODnet)",
                layers=density_layer,   # ex: un layer vessel_density YYYYMM (dépend du service)
                fmt="image/png",
                transparent=True, overlay=True, control=True, opacity=0.5,
                attr="EMODnet Human Activities"
            ).add_to(m)
    except Exception as e:
        # on n'empêche pas la carte de s'afficher si le WMS est KO
        pass


    # Marqueurs pour chaque navire
    for _, boat in df.iterrows():
        lat, lon = float(boat["latitude"]), float(boat["longitude"])
        name = str(boat["boat_name"])
        mmsi = boat.get("mmsi")
        vessel_type = boat.get("boat_type", "Unknown")
        flag_country = boat.get("flag_country", "Unknown")

        # Vérifier si c'est le navire sélectionné
        is_selected = False
        if selected_mmsi is not None and mmsi is not None:
            try:
                is_selected = (str(mmsi) == str(selected_mmsi)) or (int(mmsi) == int(selected_mmsi))
            except (ValueError, TypeError):
                is_selected = (str(mmsi) == str(selected_mmsi))

        # Style simple : sélectionné = rouge, autres = blanc
        if is_selected:
            marker_color = '#ff0000'
            border_color = '#ffffff'
            size = 12
            symbol = '★'
            opacity = 1.0
        else:
            marker_color = '#ffffff'
            border_color = '#333333'
            size = 8
            symbol = '●'
            opacity = 0.8

        # Popup avec vraies données
        popup_content = f"""
        <div style='font-family: "Courier New", monospace; font-size: 12px; min-width: 250px;'>
            <div style='background: #1a1a1a; color: #00ff41; padding: 8px; margin: -9px -9px 8px -9px; font-weight: bold;'>
                {'🔴 SÉLECTIONNÉ' if is_selected else '⚪ CONTACT'} - {name}
            </div>
            <table style='width: 100%; font-size: 11px; color: #333;'>
                <tr><td><b>MMSI:</b></td><td style='color: #0066cc;'>{mmsi}</td></tr>
                <tr><td><b>TYPE:</b></td><td>{vessel_type if vessel_type != 'Unknown' else 'Non déclaré'}</td></tr>
                <tr><td><b>PAVILLON:</b></td><td>{flag_country if flag_country != 'Unknown' else 'Non identifié'}</td></tr>
                <tr><td><b>POS:</b></td><td>{lat:.4f}°N, {lon:.4f}°E</td></tr>
                <tr><td><b>STATUS:</b></td><td style='color: #00aa00;'>TRACKED</td></tr>
            </table>
        </div>
        """

        folium.CircleMarker(
            location=[lat, lon],
            radius=size,
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=f"{symbol} {name} | MMSI {mmsi}",
            color=border_color,
            weight=2 if is_selected else 1,
            fillColor=marker_color,
            fillOpacity=opacity,
            opacity=opacity
        ).add_to(m)

        # Label pour navire sélectionné
        if is_selected:
            folium.Marker(
                [lat, lon],
                icon=folium.DivIcon(
                    html=f"""<div style='
                        font-family: "Courier New", monospace;
                        font-size: 10px;
                        font-weight: bold;
                        color: #ff0000;
                        text-shadow: 1px 1px 2px #000000;
                        background: rgba(0,0,0,0.7);
                        padding: 2px 4px;
                        border-radius: 3px;
                        white-space: nowrap;
                        transform: translate(-50%, -120%);
                    '>{name}</div>""",
                    icon_size=(1, 1),
                    icon_anchor=(0, 0)
                )
            ).add_to(m)

    folium.LayerControl().add_to(m)
    return m

print("✅ Carte créée")

✅ Carte créée


# 💻 Widgets interface base + handler sélection navire (météo/localisation/carte)

In [476]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# État global
current_boat = None
current_weather = None
current_location = None

# Mémorisation du navire sélectionné (utilisé par les tools/agent)
last_selected_mmsi = None
last_selected_name = None

# Widgets
boat_selector = widgets.Dropdown(
    options=[('Sélectionnez un bateau', None)],  # ⚠️ sera rempli dynamiquement par refresh_ui_from_stream()
    value=None,
    description='Bateau:',
    style={'description_width': 'initial'}
)

weather_output = widgets.HTML(
    value="<div style='padding:10px;background:#f0f8ff;border-radius:5px;color:#2c3e50;font-weight:500;'>Sélectionnez un bateau pour voir la météo</div>"
)

location_output = widgets.HTML(
    value="<div style='padding:10px;background:#f0fff0;border-radius:5px;color:#2c3e50;font-weight:500;'>Sélectionnez un bateau pour voir la localisation</div>"
)

map_output = widgets.Output()

# Chat (le handler sera défini plus loin en cellule 25)
chat_input = widgets.Text(
    placeholder='Posez une question sur les bateaux...',
    description='Question:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'}
)
chat_output = widgets.HTML(
    value="<div style='padding:10px;background:#fff8dc;border-radius:5px;color:#2c3e50;font-weight:500;'>💬 Chatbot prêt! Posez vos questions sur les bateaux.</div>"
)
chat_button = widgets.Button(
    description='Envoyer',
    button_style='primary',
    icon='paper-plane'
)

def update_boat_info(change):
    """Met à jour météo + localisation + carte quand un MMSI est sélectionné"""
    global current_boat, current_weather, current_location
    global last_selected_mmsi, last_selected_name

    selected_mmsi = change['new']  # valeur du Dropdown = MMSI (int/str) ou None

    # ✅ IMPORTANT: Mettre à jour la référence globale
    last_selected_mmsi = str(selected_mmsi) if selected_mmsi else None

    # Rien sélectionné
    if not selected_mmsi:
        last_selected_name = None
        weather_output.value = "<div style='padding: 10px; background: #f0f8ff; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la météo</div>"
        location_output.value = "<div style='padding: 10px; background: #f0fff0; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la localisation</div>"
        with map_output:
            clear_output(wait=True)
            try:
                live_df = vessels_state_to_df()
            except Exception:
                live_df = df_boats.copy()
            display(create_boat_map(live_df, selected_mmsi=None))
        return

    # Normaliser le MMSI sélectionné
    selected_mmsi_str = str(selected_mmsi)
    last_selected_mmsi = selected_mmsi_str

    # Chercher le navire dans le flux AIS
    v = vessels_state.get(selected_mmsi) or vessels_state.get(int(selected_mmsi)) or vessels_state.get(str(selected_mmsi))
    name = (v.get("name") or "").strip() if v else ""
    last_selected_name = name or f"MMSI {selected_mmsi_str}"

    # Coords
    lat = v.get("lat") if v else None
    lon = v.get("lon") if v else None

    # Fallback: si pas de coords côté AIS, tenter dans df_boats par nom
    if (lat is None or lon is None) and name:
        match = df_boats[df_boats['boat_name'] == name].head(1)
        if not match.empty:
            lat, lon = float(match.iloc[0]['latitude']), float(match.iloc[0]['longitude'])

    # Météo + Localisation si coords OK
    if lat is not None and lon is not None:
        weather_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Chargement météo...</div>"
        wdata = get_weather_data(lat, lon, OPENWEATHER_API_KEY)
        current_weather = wdata
        weather_info = format_weather_info(wdata)
        weather_output.value = f"<div style='padding:10px;background:#f0f8ff;border-radius:5px;white-space: pre-line;color:#2c3e50;'><h3 style='color:#34495e;margin-top:0;'>🌤️ Météo pour {last_selected_name}</h3><div style='font-weight:500;'>{weather_info}</div></div>"

        location_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Chargement localisation...</div>"
        ldata = get_location_info(lat, lon)
        current_location = ldata
        loc_info = format_location_info(ldata)
        location_output.value = f"<div style='padding:10px;background:#f0fff0;border-radius:5px;white-space: pre-line;color:#2c3e50;'><h3 style='color:#34495e;margin-top:0;'>🌍 Localisation de {last_selected_name}</h3><div style='font-weight:500;'>{loc_info}</div></div>"
    else:
        weather_output.value = "<div style='padding:10px;background:#f0f8ff;border-radius:5px;color:#2c3e50;font-weight:500;'>Coordonnées indisponibles pour ce navire pour le moment.</div>"
        location_output.value = "<div style='padding:10px;background:#f0fff0;border-radius:5px;color:#2c3e50;font-weight:500;'>Coordonnées indisponibles pour ce navire pour le moment.</div>"

    # ✅ CORRECTION PRINCIPALE: Redessiner la carte avec sélection en ROUGE
    with map_output:
        clear_output(wait=True)
        try:
            live_df = vessels_state_to_df()
            # ✅ Passer le MMSI sélectionné pour mettre l'icône en rouge
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))
        except Exception:
            live_df = df_boats.copy()
            # ✅ Fallback: créer une colonne mmsi temporaire si elle n'existe pas
            if 'mmsi' not in live_df.columns:
                live_df['mmsi'] = range(1000000, 1000000 + len(live_df))
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))

# Connecter le sélecteur (⚠️ le chat sera branché en cellule 25)
boat_selector.observe(update_boat_info, names='value')

print("✅ Interface (widgets) initialisée – la liste sera alimentée par la cellule de refresh.")



✅ Interface (widgets) initialisée – la liste sera alimentée par la cellule de refresh.


# 📜 Widgets historique discussion (output + bouton effacement)

In [477]:
history_output = widgets.HTML(
    value="<div style='padding:10px;background:#f7f8fa;border-radius:6px;color:#2c3e50;'>Aucun échange pour l'instant.</div>"
)

clear_history_button = widgets.Button(
    description='🗑️ Effacer l\'historique',
    button_style='warning',
    icon='trash',
    layout=widgets.Layout(width='200px')
)

print("✅ Widgets d'historique créés!")

✅ Widgets d'historique créés!


# 💻 Widgets interface + handlers (sélection, météo, chat, événements)

In [478]:
# Variables globales pour l'état de l'application
current_boat = None
current_weather = None
current_location = None

# ✅ Variables globales pour synchronisation avec l'agent IA
last_selected_mmsi = None
last_selected_name = None

# Widgets de l'interface
boat_selector = widgets.Dropdown(
    options=[('Sélectionnez un bateau', None)],  # ✅ Sera rempli dynamiquement par refresh_ui_from_stream()
    value=None,
    description='Bateau:',
    style={'description_width': 'initial'}
)

weather_output = widgets.HTML(
    value="<div style='padding: 10px; background: #f0f8ff; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la météo</div>"
)

location_output = widgets.HTML(
    value="<div style='padding: 10px; background: #f0fff0; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la localisation</div>"
)

map_output = widgets.Output()

# Chatbot
chat_input = widgets.Text(
    placeholder='Posez une question sur les bateaux...',
    description='Question:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'}
)

chat_output = widgets.HTML(
    value="<div style='padding: 10px; background: #fff8dc; border-radius: 5px; color: #2c3e50; font-weight: 500;'>💬 Chatbot prêt! Posez vos questions sur les bateaux.</div>"
)

chat_button = widgets.Button(
    description='Envoyer',
    button_style='primary',
    icon='paper-plane'
)

def update_boat_info(change):
    """Met à jour météo + localisation + carte quand un MMSI est sélectionné"""
    global current_boat, current_weather, current_location
    global last_selected_mmsi, last_selected_name

    selected_mmsi = change['new']  # valeur du Dropdown = MMSI (int/str) ou None

    # ✅ IMPORTANT: Mettre à jour la référence globale
    last_selected_mmsi = str(selected_mmsi) if selected_mmsi else None

    # Rien sélectionné
    if not selected_mmsi:
        last_selected_name = None
        weather_output.value = "<div style='padding: 10px; background: #f0f8ff; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la météo</div>"
        location_output.value = "<div style='padding: 10px; background: #f0fff0; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la localisation</div>"
        with map_output:
            clear_output(wait=True)
            try:
                live_df = vessels_state_to_df()
            except Exception:
                # Fallback: carte vide si pas de données AIS
                live_df = pd.DataFrame(columns=["mmsi","boat_name","latitude","longitude","boat_type","flag_country"])
            display(create_boat_map(live_df, selected_mmsi=None))
        return

    # Normaliser le MMSI sélectionné
    selected_mmsi_str = str(selected_mmsi)
    last_selected_mmsi = selected_mmsi_str

    # ✅ Chercher le navire dans le flux AIS temps réel
    v = vessels_state.get(selected_mmsi) or vessels_state.get(int(selected_mmsi)) or vessels_state.get(str(selected_mmsi))
    name = (v.get("name") or "").strip() if v else ""
    last_selected_name = name or f"MMSI {selected_mmsi_str}"

    # Coords depuis AIS
    lat = v.get("lat") if v else None
    lon = v.get("lon") if v else None

    # ✅ Pas de fallback sur df_boats (données fictives) - on utilise que les données AIS réelles
    if lat is None or lon is None:
        weather_output.value = "<div style='padding:10px;background:#f0f8ff;border-radius:5px;color:#2c3e50;font-weight:500;'>Coordonnées indisponibles pour ce navire AIS pour le moment.</div>"
        location_output.value = "<div style='padding:10px;background:#f0fff0;border-radius:5px;color:#2c3e50;font-weight:500;'>Coordonnées indisponibles pour ce navire AIS pour le moment.</div>"
        with map_output:
            clear_output(wait=True)
            try:
                live_df = vessels_state_to_df()
            except Exception:
                live_df = pd.DataFrame(columns=["mmsi","boat_name","latitude","longitude","boat_type","flag_country"])
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))
        return

    display_name = name or f"MMSI {selected_mmsi_str}"

    # Météo (OpenWeather)
    weather_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Chargement météo...</div>"
    w = get_weather_data(lat, lon, OPENWEATHER_API_KEY)
    current_weather = w
    weather_info = format_weather_info(w)
    weather_output.value = f"<div style='padding:10px;background:#f0f8ff;border-radius:5px;white-space: pre-line;color:#2c3e50;'><h3 style='color:#34495e;margin-top:0;'>🌤️ Météo pour {display_name}</h3><div style='font-weight:500;'>{weather_info}</div></div>"

    # Localisation (reverse geocoding)
    location_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Chargement localisation...</div>"
    ldata = get_location_info(lat, lon)
    current_location = ldata
    loc_info = format_location_info(ldata)
    location_output.value = f"<div style='padding:10px;background:#f0fff0;border-radius:5px;white-space: pre-line;color:#2c3e50;'><h3 style='color:#34495e;margin-top:0;'>🌍 Localisation de {display_name}</h3><div style='font-weight:500;'>{loc_info}</div></div>"

    # ✅ Redessiner la carte avec sélection en ROUGE
    with map_output:
        clear_output(wait=True)
        try:
            live_df = vessels_state_to_df()
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))
        except Exception:
            live_df = pd.DataFrame(columns=["mmsi","boat_name","latitude","longitude","boat_type","flag_country"])
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))

# ✅ Connexion des événements (le chat sera géré par la cellule handler chat avancé)
boat_selector.observe(update_boat_info, names='value')

print("✅ Interface widgets configurée (sera alimentée par le flux AIS)!")


✅ Interface widgets configurée (sera alimentée par le flux AIS)!


# 🌊 Configuration zone Méditerranée + structure données navires AIS

In [479]:
import asyncio, json, time
import nest_asyncio, websockets
import pandas as pd

nest_asyncio.apply()

# Bounding box Méditerranée (peut etre a modifier pour inclure SEULEMENT la mediterannée)
MED_MIN_LAT, MED_MAX_LAT = 29.0, 47.5     # Sud Égypte → Nord Italie/France
MED_MIN_LON, MED_MAX_LON = -6.0, 43.0     # Détroit de Gibraltar → Turquie

def in_med(lat, lon):
    if lat is None or lon is None:
        return False
    return (MED_MIN_LAT <= lat <= MED_MAX_LAT) and (MED_MIN_LON <= lon <= MED_MAX_LON)

# État courant des navires vus (clé = MMSI)
# On stocke ce qu'on sait : nom, lat, lon, sog, cog, timestamp, type, pavillon (si dispo)
vessels_state = {}

# 🆕 NOUVEAU : Dictionnaire de décodage AIS complet - DÉPLACÉ ICI
ais_type_decode = {
    # Bateaux de pêche
    30: 'Pêche', '30': 'Pêche',
    # Remorqueurs
    31: 'Remorqueur', '31': 'Remorqueur', 32: 'Remorqueur', '32': 'Remorqueur',
    # Dragueurs
    33: 'Dragueur', '33': 'Dragueur',
    # Plongée/Opérations sous-marines
    34: 'Plongée', '34': 'Plongée', 35: 'Militaire', '35': 'Militaire',
    # Voiliers
    36: 'Voilier', '36': 'Voilier', 37: 'Plaisance', '37': 'Plaisance',
    # Réservé
    38: 'Réservé', '38': 'Réservé', 39: 'Réservé', '39': 'Réservé',
    # High speed craft
    40: 'Navire rapide', '40': 'Navire rapide', 41: 'Navire rapide', '41': 'Navire rapide',
    42: 'Navire rapide', '42': 'Navire rapide', 43: 'Navire rapide', '43': 'Navire rapide',
    44: 'Navire rapide', '44': 'Navire rapide', 45: 'Navire rapide', '45': 'Navire rapide',
    46: 'Navire rapide', '46': 'Navire rapide', 47: 'Navire rapide', '47': 'Navire rapide',
    48: 'Navire rapide', '48': 'Navire rapide', 49: 'Navire rapide', '49': 'Navire rapide',
    # Pilotes
    50: 'Pilote', '50': 'Pilote', 51: 'Recherche et sauvetage', '51': 'Recherche et sauvetage',
    52: 'Remorqueur', '52': 'Remorqueur', 53: 'Tender portuaire', '53': 'Tender portuaire',
    54: 'Anti-pollution', '54': 'Anti-pollution', 55: 'Police', '55': 'Police',
    56: 'Réservé', '56': 'Réservé', 57: 'Réservé', '57': 'Réservé',
    58: 'Transport médical', '58': 'Transport médical', 59: 'Non-combattant', '59': 'Non-combattant',
    # Passagers
    60: 'Passager', '60': 'Passager', 61: 'Passager', '61': 'Passager',
    62: 'Passager', '62': 'Passager', 63: 'Passager', '63': 'Passager',
    64: 'Passager', '64': 'Passager', 65: 'Passager', '65': 'Passager',
    66: 'Passager', '66': 'Passager', 67: 'Passager', '67': 'Passager',
    68: 'Passager', '68': 'Passager', 69: 'Passager', '69': 'Passager',
    # Cargo
    70: 'Cargo', '70': 'Cargo', 71: 'Cargo', '71': 'Cargo',
    72: 'Cargo', '72': 'Cargo', 73: 'Cargo', '73': 'Cargo',
    74: 'Cargo', '74': 'Cargo', 75: 'Cargo', '75': 'Cargo',
    76: 'Cargo', '76': 'Cargo', 77: 'Cargo', '77': 'Cargo',
    78: 'Cargo', '78': 'Cargo', 79: 'Cargo', '79': 'Cargo',
    # Tankers
    80: 'Pétrolier', '80': 'Pétrolier', 81: 'Pétrolier', '81': 'Pétrolier',
    82: 'Pétrolier', '82': 'Pétrolier', 83: 'Pétrolier', '83': 'Pétrolier',
    84: 'Pétrolier', '84': 'Pétrolier', 85: 'Pétrolier', '85': 'Pétrolier',
    86: 'Pétrolier', '86': 'Pétrolier', 87: 'Pétrolier', '87': 'Pétrolier',
    88: 'Pétrolier', '88': 'Pétrolier', 89: 'Pétrolier', '89': 'Pétrolier',
    # Autres
    90: 'Autre', '90': 'Autre', 91: 'Autre', '91': 'Autre',
    92: 'Autre', '92': 'Autre', 93: 'Autre', '93': 'Autre',
    94: 'Autre', '94': 'Autre', 95: 'Autre', '95': 'Autre',
    96: 'Autre', '96': 'Autre', 97: 'Autre', '97': 'Autre',
    98: 'Autre', '98': 'Autre', 99: 'Autre', '99': 'Autre',
    # Fallbacks
    'Unknown': 'Non déclaré', None: 'Non déclaré', '': 'Non déclaré'
}

# 🆕 NOUVEAU : Dictionnaire MMSI vers pays en FRANÇAIS
mmsi_to_country = {
    '201': 'Albanie', '202': 'Andorre', '203': 'Autriche', '204': 'Açores',
    '205': 'Belgique', '206': 'Biélorussie', '207': 'Bulgarie', '208': 'Vatican',
    '209': 'Chypre', '210': 'Chypre', '211': 'Allemagne', '212': 'Chypre',
    '213': 'Géorgie', '214': 'Moldavie', '215': 'Malte', '216': 'Arménie',
    '218': 'Allemagne', '219': 'Danemark', '220': 'Danemark', '224': 'Espagne',
    '225': 'Espagne', '226': 'France', '227': 'France', '228': 'France',
    '229': 'Malte', '230': 'Finlande', '231': 'Îles Féroé', '232': 'Royaume-Uni',
    '233': 'Royaume-Uni', '234': 'Royaume-Uni', '235': 'Royaume-Uni',
    '236': 'Gibraltar', '237': 'Grèce', '238': 'Croatie', '239': 'Grèce',
    '240': 'Grèce', '241': 'Grèce', '242': 'Maroc', '243': 'Hongrie',
    '244': 'Pays-Bas', '245': 'Pays-Bas', '246': 'Pays-Bas',
    '247': 'Italie', '248': 'Malte', '249': 'Malte', '250': 'Irlande',
    '251': 'Islande', '252': 'Liechtenstein', '253': 'Luxembourg',
    '254': 'Monaco', '255': 'Madère', '256': 'Malte', '257': 'Norvège',
    '258': 'Norvège', '259': 'Norvège', '261': 'Pologne', '262': 'Monténégro',
    '263': 'Portugal', '264': 'Roumanie', '265': 'Suède', '266': 'Suède',
    '267': 'Slovaquie', '268': 'Saint-Marin', '269': 'Suisse',
    '270': 'République tchèque', '271': 'Turquie', '272': 'Ukraine',
    '273': 'Russie', '274': 'Macédoine', '275': 'Lettonie', '276': 'Estonie',
    '277': 'Lituanie', '278': 'Slovénie', '279': 'Serbie', '301': 'Anguilla',
    '303': 'Alaska', '304': 'Antigua-et-Barbuda', '305': 'Antigua-et-Barbuda',
    '306': 'Antilles néerlandaises', '307': 'Aruba', '308': 'Bahamas',
    '309': 'Bahamas', '310': 'Bermudes', '311': 'Bahamas', '312': 'Belize',
    '314': 'Barbade', '316': 'Canada', '319': 'Îles Caïmans',
    '321': 'Costa Rica', '323': 'Cuba', '325': 'Dominique', '327': 'République dominicaine',
    '329': 'Guadeloupe', '330': 'Grenade', '331': 'Groenland', '332': 'Guatemala',
    '334': 'Honduras', '336': 'Haïti', '338': 'États-Unis', '339': 'Jamaïque',
    '341': 'Saint-Kitts-et-Nevis', '343': 'Sainte-Lucie', '345': 'Mexique',
    '347': 'Martinique', '348': 'Montserrat', '350': 'Nicaragua',
    '351': 'Panama', '352': 'Panama', '353': 'Panama', '354': 'Panama',
    '355': 'Panama', '356': 'Panama', '357': 'Panama', '358': 'Porto Rico',
    '359': 'Salvador', '361': 'Saint-Pierre-et-Miquelon',
    '362': 'Trinité-et-Tobago', '364': 'Îles Turques-et-Caïques',
    '366': 'États-Unis', '367': 'États-Unis', '368': 'États-Unis',
    '369': 'États-Unis', '370': 'Panama', '371': 'Panama', '372': 'Panama',
    '373': 'Panama', '374': 'Panama', '375': 'Saint-Vincent-et-les-Grenadines',
    '376': 'Saint-Vincent-et-les-Grenadines', '377': 'Saint-Vincent-et-les-Grenadines',
    '378': 'Îles Vierges britanniques', '379': 'Îles Vierges américaines',
    '401': 'Afghanistan', '403': 'Arabie saoudite', '405': 'Bangladesh',
    '408': 'Bahreïn', '410': 'Bhoutan', '412': 'Chine', '413': 'Chine',
    '414': 'Chine', '416': 'Taïwan', '417': 'Sri Lanka', '419': 'Inde',
    '422': 'Iran', '423': 'Azerbaïdjan', '425': 'Irak', '428': 'Israël',
    '431': 'Japon', '432': 'Japon', '434': 'Turkménistan', '436': 'Kazakhstan',
    '437': 'Ouzbékistan', '438': 'Jordanie', '440': 'Corée du Sud', '441': 'Corée du Sud',
    '443': 'Palestine', '445': 'Corée du Nord', '447': 'Koweït', '450': 'Liban',
    '451': 'Kirghizistan', '453': 'Macao', '455': 'Maldives', '457': 'Mongolie',
    '459': 'Népal', '461': 'Oman', '463': 'Pakistan', '466': 'Qatar',
    '468': 'Syrie', '470': 'Émirats arabes unis', '472': 'Tadjikistan',
    '473': 'Yémen', '475': 'Yémen', '477': 'Hong Kong', '478': 'Bosnie-Herzégovine',
    '501': 'France', '503': 'Australie', '506': 'Myanmar', '508': 'Brunei',
    '510': 'Micronésie', '511': 'Palaos', '512': 'Nouvelle-Zélande', '514': 'Cambodge',
    '515': 'Cambodge', '516': 'Île Christmas', '518': 'Îles Cook',
    '520': 'Fidji', '523': 'Îles Cocos', '525': 'Indonésie', '529': 'Kiribati',
    '531': 'Laos', '533': 'Malaisie', '536': 'Îles Mariannes du Nord',
    '538': 'Îles Marshall', '540': 'Nouvelle-Calédonie', '542': 'Niue',
    '544': 'Nauru', '546': 'Polynésie française', '548': 'Philippines',
    '553': 'Papouasie-Nouvelle-Guinée', '555': 'Île Pitcairn', '557': 'Îles Solomon',
    '559': 'Samoa américaines', '561': 'Samoa', '563': 'Singapour', '564': 'Singapour',
    '565': 'Singapour', '566': 'Singapour', '567': 'Thaïlande', '570': 'Tonga',
    '572': 'Tuvalu', '574': 'Vietnam', '576': 'Vanuatu', '577': 'Vanuatu',
    '578': 'Wallis-et-Futuna', '601': 'Afrique du Sud', '603': 'Angola',
    '605': 'Algérie', '607': 'Îles Saint-Paul et Amsterdam', '608': 'Île de l\'Ascension',
    '609': 'Burundi', '610': 'Bénin', '611': 'Botswana', '612': 'République centrafricaine',
    '613': 'Cameroun', '615': 'Congo', '616': 'Comores', '617': 'Cap-Vert',
    '618': 'Archipel Crozet', '619': 'Côte d\'Ivoire', '620': 'Comores',
    '621': 'Djibouti', '622': 'Égypte', '624': 'Éthiopie', '625': 'Érythrée',
    '626': 'Gabon', '627': 'Ghana', '629': 'Gambie',
    '630': 'Guinée-Bissau', '631': 'Guinée équatoriale', '632': 'Guinée',
    '633': 'Burkina Faso', '634': 'Kenya', '635': 'Îles Kerguelen',
    '636': 'Libéria', '637': 'Libéria', '638': 'Soudan du Sud', '642': 'Libye',
    '644': 'Lesotho', '645': 'Maurice', '647': 'Madagascar', '649': 'Mali',
    '650': 'Mozambique', '654': 'Mauritanie', '655': 'Malawi', '656': 'Niger',
    '657': 'Nigeria', '659': 'Namibie', '660': 'Réunion', '661': 'Rwanda',
    '662': 'Soudan', '663': 'Sénégal', '664': 'Seychelles', '665': 'Sainte-Hélène',
    '666': 'Somalie', '667': 'Sierra Leone', '668': 'São Tomé-et-Príncipe',
    '669': 'Eswatini', '670': 'Tchad', '671': 'Togo',
    '672': 'Tunisie', '674': 'Tanzanie', '675': 'Ouganda', '676': 'République démocratique du Congo',
    '677': 'Tanzanie', '678': 'Zambie', '679': 'Zimbabwe'
}


def vessels_state_to_df():
    """Convertit les données AIS en DataFrame avec décodage amélioré, filtré par Méditerranée"""

    rows = []

    for mmsi_str, v in vessels_state.items():
        try:
            mmsi = int(mmsi_str)
            lat = float(v.get("lat", 0))
            lon = float(v.get("lon", 0))

            # Filtrer les navires qui ne sont PAS dans la Méditerranée
            if not in_med(lat, lon):
                continue # Sauter ce navire s'il est hors zone

            # Décodage du type de navire
            raw_type = v.get("type")
            decoded_type = "Non déclaré"
            if raw_type is not None and str(raw_type).strip():
                try:
                    # Essayer conversion en int d'abord
                    decoded_type = ais_type_decode.get(int(raw_type), "Non déclaré")
                except (ValueError, TypeError):
                    # Si échec, essayer en string
                    decoded_type = ais_type_decode.get(str(raw_type), "Non déclaré")

            # Décodage du pays depuis MMSI
            mmsi_prefix = str(mmsi)[:3]
            country = mmsi_to_country.get(mmsi_prefix, "Non identifié")

            rows.append({
                "mmsi": mmsi,
                "boat_name": v.get("name", f"MMSI_{mmsi}"),
                "latitude": lat,
                "longitude": lon,
                "boat_type": decoded_type,
                "flag_country": country
            })

        except (ValueError, TypeError, KeyError) as e:
            continue

    return pd.DataFrame(rows)

# 🔍 Helper résolution coordonnées navire sélectionné (nom/MMSI → lat/lon)

In [480]:
# 🔎 Helper pour retrouver lat/lon d'un bateau sélectionné depuis le flux AIS

def resolve_selection_to_coords(selection_label):
    """
    Retrouve (lat, lon, name) d'un bateau sélectionné en se basant sur vessels_state (AIS réel).
    - selection_label est la valeur du Dropdown (actuellement le nom ou "MMSI NNN...")
    - retourne (lat, lon, name) ou (None, None, None) si non trouvé
    """
    if not selection_label:
        return None, None, None

    # 1) correspondance exacte sur le nom reçu dans ShipStaticData
    for mmsi, v in vessels_state.items():
        name = (v.get("name") or "").strip()
        if name and name == selection_label:
            return v.get("lat"), v.get("lon"), name

    # 2) correspondance sur libellé "MMSI 123456789" si présent dans la liste
    if selection_label.startswith("MMSI "):
        try:
            mmsi_int = int(selection_label.split("MMSI ", 1)[1])
            v = vessels_state.get(mmsi_int) or vessels_state.get(str(mmsi_int))
            if v and v.get("lat") is not None and v.get("lon") is not None:
                return v.get("lat"), v.get("lon"), v.get("name") or selection_label
        except Exception:
            pass

    # 3) pas trouvé côté AIS
    return None, None, None


# 🛰️ Client WebSocket AISStream avec gestion d'erreurs robuste

In [481]:
async def aisstream_connect_and_consume():
    """
    Connexion au WebSocket AISStream avec gestion d'erreurs robuste
    """
    uri = "wss://stream.aisstream.io/v0/stream"

    try:
        async with websockets.connect(
            uri,
            ping_interval=20,
            ping_timeout=20,
            close_timeout=10,
            max_size=2**20  # 1MB max message size
        ) as ws:

            # Message de souscription
            subscribe_msg = {
                "APIKey": AISSTREAM_API_KEY,
                "BoundingBoxes": [[[MED_MIN_LAT, MED_MIN_LON], [MED_MAX_LAT, MED_MAX_LON]]],
                "FilterMessageTypes": ["PositionReport", "ShipStaticData"]
            }

            await ws.send(json.dumps(subscribe_msg))
            print("✅ Souscription AISStream envoyée")

            message_count = 0
            async for raw in ws:
                try:
                    msg = json.loads(raw)
                    message_count += 1

                    # Log périodique
                    if message_count % 100 == 0:
                        print(f"📊 {message_count} messages AIS reçus, {len(vessels_state)} navires en mémoire")

                    mtype = msg.get("MessageType")
                    data = msg.get("Message", {})

                    # Position des navires
                    if mtype == "PositionReport":
                        pr = data.get("PositionReport", {})
                        mmsi = pr.get("UserID")
                        lat = pr.get("Latitude")
                        lon = pr.get("Longitude")
                        sog = pr.get("Sog")
                        cog = pr.get("Cog")

                        if mmsi and lat is not None and lon is not None and in_med(lat, lon):
                            v = vessels_state.setdefault(mmsi, {})
                            v.update({
                                "lat": lat, "lon": lon, "sog": sog, "cog": cog,
                                "last_ts": time.time()
                            })

                        ts_now = time.time()
                        _append_history(mmsi=mmsi, ts=ts_now, lat=lat, lon=lon, sog=sog, cog=cog)

                    # Données statiques
                    elif mtype == "ShipStaticData":
                        sd = data.get("ShipStaticData", {})
                        mmsi = sd.get("UserID")
                        name = (sd.get("Name") or "").strip()
                        stype = sd.get("Type")

                        if mmsi:
                            v = vessels_state.setdefault(mmsi, {})
                            if name:
                                v["name"] = name
                            if stype:
                                v["type"] = str(stype)

                except json.JSONDecodeError:
                    continue  # Ignorer les messages malformés
                except Exception as e:
                    print(f"⚠️ Erreur traitement message AIS: {e}")
                    continue

    except websockets.exceptions.ConnectionClosedError as e:
        print(f"🔌 Connexion WebSocket fermée: {e}")
        raise
    except Exception as e:
        print(f"❌ Erreur connexion AISStream: {e}")
        raise

print("✅ Client WebSocket AISStream configuré avec gestion d'erreurs")



✅ Client WebSocket AISStream configuré avec gestion d'erreurs


# ▶️ Démarrage flux AISStream avec reconnexion automatique et gestion d'erreurs

In [482]:
import asyncio
import time

# Variable globale pour contrôler la tâche
stream_task = None

async def aisstream_connect_with_retry():
    """
    Lance le flux AISStream avec gestion d'erreurs et reconnexion automatique
    """
    retry_count = 0
    max_retries = 5
    base_delay = 5  # secondes

    while retry_count < max_retries:
        try:
            print(f"🔄 Tentative de connexion AISStream #{retry_count + 1}")
            await aisstream_connect_and_consume()
            # Si on arrive ici, la connexion s'est fermée normalement
            print("⚠️ Connexion AISStream fermée normalement")
            break

        except Exception as e:
            retry_count += 1
            error_msg = str(e)

            if "ConnectionClosedError" in error_msg:
                print(f"⚠️ Connexion AISStream fermée (tentative {retry_count}/{max_retries})")
            elif "TimeoutError" in error_msg:
                print(f"⏰ Timeout AISStream (tentative {retry_count}/{max_retries})")
            else:
                print(f"❌ Erreur AISStream: {error_msg} (tentative {retry_count}/{max_retries})")

            if retry_count < max_retries:
                delay = base_delay * (2 ** (retry_count - 1))  # Backoff exponentiel
                print(f"⏳ Reconnexion dans {delay}s...")
                await asyncio.sleep(delay)
            else:
                print("❌ Nombre maximum de tentatives atteint. Arrêt du flux AIS.")
                break

def start_ais_stream():
    """
    Démarre le flux AISStream de manière sécurisée
    """
    global stream_task

    # Arrêter la tâche précédente si elle existe
    if stream_task and not stream_task.done():
        stream_task.cancel()
        print("🛑 Arrêt de la tâche AIS précédente")

    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

    # Créer la nouvelle tâche avec gestion d'erreurs
    stream_task = loop.create_task(aisstream_connect_with_retry())

    # Ajouter un callback pour gérer les exceptions non récupérées
    def handle_task_exception(task):
        if task.exception():
            print(f"⚠️ Tâche AIS terminée avec exception: {task.exception()}")
        else:
            print("✅ Tâche AIS terminée normalement")

    stream_task.add_done_callback(handle_task_exception)
    print("🚀 Flux AISStream lancé avec gestion d'erreurs (zone Méditerranée)")

# Démarrer le flux
start_ais_stream()


🚀 Flux AISStream lancé avec gestion d'erreurs (zone Méditerranée)


# 🔄 Fonctions refresh interface (dropdown + carte) depuis flux AIS

In [483]:
from IPython.display import clear_output, display

def refresh_ui_from_stream(selected_mmsi=None, max_ships=300):
    live_df = vessels_state_to_df()

    # Même si vide : on affiche la carte (avec EMODnet) et un message
    if live_df.empty:
        with map_output:
            clear_output(wait=True)
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi))
        print("⏳ En attente de premiers messages AIS…")
        return

    if len(live_df) > max_ships:
        live_df = live_df.head(max_ships)

    # Options du Dropdown: étiquette lisible, valeur = MMSI (unique)
    def labelize(row):
        n = (row['boat_name'] or '').strip()
        mm = row['mmsi']
        return f"{n} — MMSI {mm}" if n else f"MMSI {mm}"

    names = [ (labelize(r), r['mmsi']) for _, r in live_df.sort_values('boat_name').iterrows() ]
    new_options = [('Sélectionnez un bateau', None)] + names

    try:
        boat_selector.options = new_options
        if selected_mmsi is not None and any(v == selected_mmsi for _, v in new_options):
            boat_selector.value = selected_mmsi
        else:
            boat_selector.value = None
    except Exception:
        pass

    with map_output:
        clear_output(wait=True)
        display(create_boat_map(live_df, selected_mmsi=selected_mmsi))

def run_refresh_loop(seconds=10, iterations=30):
    """
    Rafraîchir automatiquement toutes les X secondes pendant N itérations.
    """
    sel = None
    for i in range(iterations):
        print(f"🔄 Refresh {i+1}/{iterations}")
        refresh_ui_from_stream(selected_mmsi=sel)
        time.sleep(seconds)
        clear_output(wait=True)
    print("✅ Boucle de refresh terminée.")

In [484]:
def history_debug(mmsi: str, minutes=60):
    key = str(mmsi)
    series = list(ais_history.get(key, []))
    print(f"🔎 {key} → {len(series)} points en mémoire courte")
    if series:
        print("  Dernier point:", series[-1])
    cutoff = time.time() - minutes*60
    recent = [p for p in series if p["ts"] >= cutoff]
    print(f"  → {len(recent)} points dans les {minutes} dernières minutes")

def tail_archive(day_offset=0, n=5):
    import datetime as _dt, os
    day = (_dt.datetime.utcnow() - _dt.timedelta(days=day_offset)).strftime("%Y%m%d")
    path = f"{ARCHIVE_DIR}/positions_{day}.jsonl"
    if not os.path.exists(path):
        print("❌ Aucune archive pour", path)
        return
    print("📄", path, "(last lines)")
    with open(path, "rb") as f:
        try:
            f.seek(-20000, 2)  # remonte ~20KB depuis la fin
        except OSError:
            f.seek(0, 0)
        lines = f.read().decode("utf-8", errors="ignore").strip().splitlines()
        for line in lines[-n:]:
            print(line)


In [485]:
def seed_history_from_snapshot(max_age_sec=3600):
    """Sème 1 point dans l'historique pour tout navire ayant (lat, lon) en mémoire,
       daté de son last_ts si présent, sinon 'maintenant'."""
    now = time.time()
    c = 0
    for mm, vv in vessels_state.items():
        lat, lon = vv.get("lat"), vv.get("lon")
        if lat is None or lon is None:
            continue
        ts = float(vv.get("last_ts") or now)
        # optionnel: filtrer sur fraicheur
        if (now - ts) > max_age_sec:
            continue
        sog = vv.get("sog")
        cog = vv.get("cog")
        _append_history(mmsi=mm, ts=ts, lat=lat, lon=lon, sog=sog, cog=cog)
        c += 1
    print(f"✅ Seed: {c} points injectés depuis le snapshot.")

# Lance le seed une fois :
seed_history_from_snapshot(max_age_sec=6*3600)


✅ Seed: 0 points injectés depuis le snapshot.


# ⏰ Premier refresh interface + refresh manuel (à relancer pour actualiser)

In [486]:
# Attendre que le flux AIS commence à recevoir des données
time.sleep(5)

# Premier refresh pour afficher les navires détectés
refresh_ui_from_stream()

print("✅ Premier refresh effectué")
print("📝 IMPORTANT: Pour actualiser la liste des navires, relancez cette cellule")
print("   ou utilisez la boucle automatique ci-dessous")

# OU lancer une boucle auto (toutes les 10s, 30 fois) :
# run_refresh_loop(seconds=10, iterations=30)

print("\n🔄 Options de refresh :")
print("• Relancer cette cellule manuellement quand vous voulez actualiser")
print("• Décommenter la ligne run_refresh_loop() pour un refresh automatique")
print("• Les nouveaux navires AIS apparaîtront dans le dropdown après refresh")


⏳ En attente de premiers messages AIS…
✅ Premier refresh effectué
📝 IMPORTANT: Pour actualiser la liste des navires, relancez cette cellule
   ou utilisez la boucle automatique ci-dessous

🔄 Options de refresh :
• Relancer cette cellule manuellement quand vous voulez actualiser
• Décommenter la ligne run_refresh_loop() pour un refresh automatique
• Les nouveaux navires AIS apparaîtront dans le dropdown après refresh


# 🧰 Fonctions utilitaires (Haversine, recherche navires, météo, bathymétrie, Beaufort)

In [487]:
import math, requests, difflib, unicodedata, datetime as dt

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2 * R * math.asin(math.sqrt(a))

def _norm(s: str) -> str:
    if not s: return ""
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return s.lower().strip()

def vessel_candidates(query: str, k: int = 10):
    qn = _norm(query or "")
    by_name = []
    for mmsi, v in vessels_state.items():
        name = (v.get("name") or "").strip()
        if not name: continue
        nn = _norm(name)
        sub = (qn in nn) if qn else False
        ratio = difflib.SequenceMatcher(None, qn, nn).ratio() if qn else 0
        score = 1.0 if sub else ratio
        by_name.append((score, str(mmsi), name))
    by_name.sort(reverse=True, key=lambda x: x[0])
    return [(m, n, s) for s, m, n in by_name[:k]]

def resolve_vessel_query(query: str, prefer_mmsi: str = None):
    """Essaie d'abord le MMSI sélectionné, sinon la requête (MMSI ou nom approx)."""
    # 1) MMSI favori
    if prefer_mmsi:
        v = vessels_state.get(int(prefer_mmsi)) or vessels_state.get(str(prefer_mmsi))
        if v and v.get("lat") is not None and v.get("lon") is not None:
            name = (v.get("name") or "").strip() or f"MMSI {prefer_mmsi}"
            return str(prefer_mmsi), v, name

    # 2) MMSI repéré dans la requête
    import re
    m = re.search(r'(\d{7,9})', str(query or ""))
    if m:
        mm = m.group(1)
        v = vessels_state.get(int(mm)) or vessels_state.get(mm)
        if v and v.get("lat") is not None and v.get("lon") is not None:
            name = (v.get("name") or "").strip() or f"MMSI {mm}"
            return str(mm), v, name

    # 3) nom approx
    qn = _norm(query or "")
    idx = {}
    for mmsi, v in vessels_state.items():
        name = (v.get("name") or "").strip()
        if name: idx[_norm(name)] = (str(mmsi), v, name)
    if qn in idx: return idx[qn]
    for nn, triple in idx.items():
        if qn and qn in nn: return triple
    cands = vessel_candidates(query, k=1)
    if cands:
        mmsi, name, _s = cands[0]
        v = vessels_state.get(int(mmsi)) or vessels_state.get(str(mmsi))
        if v: return str(mmsi), v, name

    return None, None, None

# --- OpenWeather current (lat/lon)
def openweather_current(lat: float, lon: float, api_key: str):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"lat": lat, "lon": lon, "appid": api_key, "units": "metric", "lang": "fr"}
    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    return r.json()

# --- EMODnet Bathymetry REST: depth at point (WKT POINT(lon lat))
def emodnet_depth_sample(lat: float, lon: float):
    wkt = f"POINT({lon} {lat})"
    url = "https://rest.emodnet-bathymetry.eu/depth_sample"
    r = requests.get(url, params={"geom": wkt}, timeout=10)
    r.raise_for_status()
    return r.json()  # ex: {"min":31.2,"max":31.3,"avg":31.25,"smoothed":30.95,...}

# --- Beaufort
def bft_from_ms(v_ms: float) -> int:
    if v_ms is None: return None
    bounds = [0.3,1.6,3.4,5.5,8.0,10.8,13.9,17.2,20.8,24.5,28.5,32.7]
    for i,t in enumerate(bounds):
        if v_ms < t: return i
    return 12

def bft_label(b: int) -> str:
    labels = {
        0:"Calme",1:"Très légère brise",2:"Légère brise",3:"Brise légère",
        4:"Petite brise",5:"Bonne brise",6:"Vent frais",7:"Grand frais",
        8:"Coup de vent",9:"Fort coup de vent",10:"Tempête",11:"Violente tempête",12:"Ouragan"
    }
    return labels.get(b, "")



In [488]:
import glob, datetime as dt

def _load_archive_points(mmsi: str, since_ts: float):
    """Charge depuis /content/ais_archive/* les points >= since_ts pour ce MMSI."""
    key = str(mmsi)
    out = []
    # aujourd'hui + hier (suffisant pour la plupart des tests)
    days = [dt.datetime.utcnow().strftime("%Y%m%d"),
            (dt.datetime.utcnow() - dt.timedelta(days=1)).strftime("%Y%m%d")]
    for day in days:
        path = f"{ARCHIVE_DIR}/positions_{day}.jsonl"
        try:
            with open(path, "r", encoding="utf-8") as f:
                for line in f:
                    try:
                        rec = json.loads(line)
                        if rec.get("mmsi") == key and rec.get("ts", 0) >= since_ts:
                            out.append(rec)
                    except Exception:
                        continue
        except FileNotFoundError:
            continue
    # tri par temps croissant
    out.sort(key=lambda r: r["ts"])
    return out


In [489]:
# === [NOUVELLE CELLULE] Tool de détection d'anomalies ===
from langchain_core.tools import tool
import math

def _ang_diff(a,b):
    """Différence angulaire normalisée [-180,180]."""
    if a is None or b is None: return None
    d = (a - b + 180) % 360 - 180
    return d

def _path_metrics(points):
    """
    points: liste triée par ts asc de dicts {ts,lat,lon,sog,cog}
    Retourne métriques utiles: longueur parcourue, rectitude, somme des virages, fraction <4kn, gaps, etc.
    """
    if len(points) < 3:
        return {"ok": False, "reason":"not_enough_points"}

    total_m = 0.0
    total_turn = 0.0
    below4 = 0
    gaps = []
    first = points[0]; last = points[-1]

    for i in range(1, len(points)):
        p0, p1 = points[i-1], points[i]
        # distance
        try:
            dm = haversine_m(p0["lat"], p0["lon"], p1["lat"], p1["lon"])
            total_m += dm
        except Exception:
            pass
        # virage
        dth = _ang_diff(p1.get("cog"), p0.get("cog"))
        if dth is not None:
            total_turn += abs(dth)
        # vitesses lentes
        if p1.get("sog") is not None and p1["sog"] < 4.0:
            below4 += 1
        # trous temporels
        if p1["ts"] - p0["ts"] > 15*60:  # >15 min sans point AIS
            gaps.append(p1["ts"] - p0["ts"])

    # “rectitude”: distance orthodromique / distance parcourue
    try:
        gc_m = haversine_m(first["lat"], first["lon"], last["lat"], last["lon"])
        straightness_ratio = (total_m / gc_m) if gc_m > 1.0 else float("inf")
    except Exception:
        gc_m, straightness_ratio = None, None

    frac_below4 = below4 / max(1, len(points))
    duration_min = (last["ts"] - first["ts"]) / 60.0

    return {
        "ok": True,
        "n": len(points),
        "duration_min": duration_min,
        "track_len_m": total_m,
        "gc_m": gc_m,
        "straightness_ratio": straightness_ratio,   # > ~1.8-2.0 => erratique/circulaire
        "total_turn_deg": total_turn,               # > ~360-720° sur 30-60min => loitering
        "frac_below4": frac_below4,                 # > ~0.6 => lenteur persistante
        "gaps_sec": gaps
    }

# 🛠️ Tools LangChain (ais_lookup + risk_assess) pour agent IA

In [490]:
from langchain_core.tools import tool
import math
import numpy as np # Import numpy
import time # Import time for historical data checks

def _ang_diff(a,b):
    """Différence angulaire normalisée [-180,180]."""
    if a is None or b is None: return None
    d = (a - b + 180) % 360 - 180
    return d

def _path_metrics(points):
    """
    points: liste triée par ts asc de dicts {ts,lat,lon,sog,cog}
    Retourne métriques utiles: longueur parcourue, rectitude, somme des virages, fraction <4kn, gaps, etc.
    """
    if len(points) < 3:
        return {"ok": False, "reason":"not_enough_points"}

    total_m = 0.0
    total_turn = 0.0
    below4 = 0
    gaps = []
    first = points[0]; last = points[-1]

    for i in range(1, len(points)):
        p0, p1 = points[i-1], points[i]
        # distance
        try:
            dm = haversine_m(p0["lat"], p0["lon"], p1["lat"], p1["lon"])
            total_m += dm
        except Exception:
            pass
        # virage
        dth = _ang_diff(p1.get("cog"), p0.get("cog"))
        if dth is not None:
            total_turn += abs(dth)
        # vitesses lentes
        if p1.get("sog") is not None and p1["sog"] < 4.0:
            below4 += 1
        # trous temporels
        if p1["ts"] - p0["ts"] > 15*60:  # >15 min sans point AIS
            gaps.append(p1["ts"] - p0["ts"])

    # “rectitude”: distance orthodromique / distance parcourue
    try:
        gc_m = haversine_m(first["lat"], first["lon"], last["lat"], last["lon"])
        straightness_ratio = (total_m / gc_m) if gc_m > 1.0 else float("inf")
    except Exception:
        gc_m, straightness_ratio = None, None

    frac_below4 = below4 / max(1, len(points))
    duration_min = (last["ts"] - first["ts"]) / 60.0

    return {
        "ok": True,
        "n": len(points),
        "duration_min": duration_min,
        "track_len_m": total_m,
        "gc_m": gc_m,
        "straightness_ratio": straightness_ratio,   # > ~1.8-2.0 => erratique/circulaire
        "total_turn_deg": total_turn,               # > ~360-720° sur 30-60min => loitering
        "frac_below4": frac_below4,                 # > ~0.6 => lenteur persistante
        "gaps_sec": gaps
    }

@tool
def ais_lookup(vessel_query: str, selected_mmsi_hint: str = ""):
    """
    Trouve un navire par NOM/MMSI.
    Essaie d'abord le MMSI sélectionné dans l'UI (selected_mmsi_hint), puis la requête utilisateur.
    Met à jour 'last_ref' si trouvé.
    Retourne les informations clés du navire, y compris le type et le pavillon si disponibles.
    """
    global last_ref
    hint = selected_mmsi_hint or (last_selected_mmsi or "")
    mmsi, v, name = resolve_vessel_query(vessel_query, prefer_mmsi=hint)
    if not v:
        cands = vessel_candidates(vessel_query, k=5)
        return {"status": "not_found", "candidates": [{"mmsi": m, "name": n} for (m, n, _s) in cands]}

    lat, lon = v.get("lat"), v.get("lon")
    last_ref = {"mmsi": str(mmsi), "name": name, "lat": lat, "lon": lon}

    # Decode type and country using the same logic as vessels_state_to_df
    raw_type = v.get("type")
    decoded_type = "Non déclaré"
    if raw_type is not None and str(raw_type).strip():
        try:
            # Access ais_type_decode directly now that it's global in cell xI1-IFkJibSK
            decoded_type = ais_type_decode.get(int(raw_type), "Non déclaré")
        except (ValueError, TypeError):
            # Access ais_type_decode directly now that it's global in cell xI1-IFkJibSK
            decoded_type = ais_type_decode.get(str(raw_type), "Non déclaré")

    mmsi_prefix = str(mmsi)[:3]
    # Access mmsi_to_country directly now that it's global in cell xI1-IFkJibSK
    country = mmsi_to_country.get(mmsi_prefix, "Non identifié")


    return {
        "status": "ok",
        "mmsi": str(mmsi),
        "name": name,
        "lat": lat, "lon": lon,
        "sog_kn": v.get("sog"),
        "cog_deg": v.get("cog"),
        "draught_m": (0.1*float(v.get("draught"))) if (v.get("draught") not in (None, 0, 255)) else None,
        "type": decoded_type, # Include type
        "flag_country": country # Include flag
    }

@tool
def risk_assess(vessel_query: str = "", neighbor_radius_nm: float = 2.0):
    """
    Évalue le risque maritime pour un navire donné.
    Analyse: conditions météo, profondeur, houle, trafic local, tirant d'eau si connu.
    Met à jour 'last_ref' si trouvé.
    """
    global last_ref
    mmsi, v, name = resolve_vessel_query(vessel_query, prefer_mmsi=last_selected_mmsi)
    if not v:
        return {"status": "not_found", "candidates": [{"mmsi": m, "name": n} for (m, n, _s) in vessel_candidates(vessel_query, k=5)]}

    lat, lon = v.get("lat"), v.get("lon")
    if lat is None or lon is None:
        return {"status": "no_coords", "message": "Coordonnées indisponibles pour ce navire"}

    last_ref = {"mmsi": str(mmsi), "name": name, "lat": lat, "lon": lon}

    try:
        # --- Météo avec gestion d'erreur ---
        wind, bft = None, None
        try:
            w = openweather_current(lat, lon, OPENWEATHER_API_KEY)
            wind = w.get("wind", {}).get("speed")  # m/s
            bft = bft_from_ms(wind) if wind is not None else None
        except Exception as e:
            print(f"Erreur météo: {e}")

        # --- Bathymétrie avec gestion d'erreur ---
        depth_m = None
        try:
            bathy = emodnet_depth_sample(lat, lon)
            raw = bathy.get("avg") or bathy.get("smoothed")
            depth_m = abs(float(raw)) if raw is not None else None
        except Exception as e:
            print(f"Erreur bathymétrie: {e}")

        # --- Houle (simulation simple) ---
        swh = None
        # Note: Pour la houle, on pourra intégrer Copernicus Marine plus tard

        # --- Analyse du trafic local ---
        neighbors_details = []
        for mm, vv in vessels_state.items():
            if str(mm) == str(mmsi):
                continue
            la, lo = vv.get("lat"), vv.get("lon")
            if la is None or lo is None:
                continue

            try:
                rng_m = haversine_m(lat, lon, la, lo)
                if rng_m <= neighbor_radius_nm * 1852.0:
                    neighbors_details.append({
                        "mmsi": str(mm),
                        "name": (vv.get("name") or "").strip() or f"MMSI {mm}",
                        "range_nm": round(rng_m / 1852.0, 2),
                        "sog": vv.get("sog"),
                        "cog": vv.get("cog")
                    })
            except Exception:
                continue

        neighbors_details.sort(key=lambda x: x["range_nm"])
        neighbors = len(neighbors_details)
        neighbors_top5 = neighbors_details[:5]

        # --- Calcul du tirant d'eau et UKC ---
        draught_m = None
        try:
            dr = v.get("draught")
            if dr not in (None, 0, 255):
                draught_m = 0.1 * float(dr)
        except Exception:
            pass

        ukc = None
        if depth_m is not None and draught_m is not None:
            ukc = depth_m - draught_m - 0.5  # Marge de sécurité de 0.5m

        # --- Calcul du score de risque ---
        score, notes = 0, []

        # Risque météorologique
        if bft is not None:
            if bft >= 8:
                score += 3
                notes.append(f"Vent très fort Beaufort {bft} - Conditions dangereuses")
            elif bft >= 6:
                score += 2
                notes.append(f"Vent fort Beaufort {bft} - Navigation difficile")
            elif bft >= 5:
                score += 1
                notes.append(f"Vent modéré Beaufort {bft} - Vigilance requise")

        # Risque de profondeur/échouage
        if ukc is not None:
            if ukc < 0.5:
                score += 4
                notes.append(f"UKC critique {ukc:.1f}m - Risque d'échouage imminent")
            elif ukc < 1.0:
                score += 3
                notes.append(f"UKC très faible {ukc:.1f}m - Risque d'échouage élevé")
            elif ukc < 2.0:
                score += 2
                notes.append(f"UKC faible {ukc:.1f}m - Navigation délicate")
            elif ukc < 3.0:
                score += 1
                notes.append(f"UKC modérée {ukc:.1f}m - Prudence recommandée")
        elif depth_m is not None and draught_m is None:
            if depth_m < 3:
                score += 3
                notes.append(f"Très faible fond {depth_m:.1f}m - Tirant inconnu, risque d'échouage")
            elif depth_m < 10:
                score += 2
                notes.append(f"Faible fond {depth_m:.1f}m - Tirant inconnu, prudence requise")
            elif depth_m < 15:
                score += 1
                notes.append(f"Fond modéré {depth_m:.1f}m - Tirant inconnu, surveillance recommandée")

        # Risque de trafic
        if neighbors >= 15:
            score += 2
            notes.append(f"Trafic très dense ({neighbors} navires ≤{neighbor_radius_nm}nm) - Risque de collision")
        elif neighbors >= 10:
            score += 2
            notes.append(f"Trafic dense ({neighbors} navires ≤{neighbor_radius_nm}nm) - Surveillance radar renforcée")
        elif neighbors >= 5:
            score += 1
            notes.append(f"Trafic modéré ({neighbors} navires ≤{neighbor_radius_nm}nm) - Veille normale")

        # Détermination du niveau de risque
        if score >= 6:
            level = "critique"
        elif score >= 4:
            level = "élevé"
        elif score >= 2:
            level = "modéré"
        else:
            level = "faible"

        return {
            "status": "ok",
            "mmsi": str(mmsi),
            "name": name,
            "lat": lat,
            "lon": lon,
            "wind_mps": wind,
            "beaufort": bft,
            "depth_m_LAT": depth_m,
            "swh_m": swh,
            "neighbors": neighbors,
            "neighbors_top5": neighbors_top5,
            "draught_m": draught_m,
            "ukc_m": ukc,
            "risk_score": score,
            "risk_level": level,
            "notes": notes
        }

    except Exception as e:
        return {
            "status": "error",
            "message": f"Erreur lors de l'évaluation de risque: {str(e)}",
            "mmsi": str(mmsi) if mmsi else None,
            "name": name if name else None
        }

@tool
def detect_anomalies(vessel_query: str, minutes: int = 240):
    """
    Détecte des comportements suspects sur la base de l'historique AIS sur une période donnée (par défaut 240 minutes):
      - Vitesse anormalement basse (< 5 kn) sur trajectoire non rectiligne (loitering)
      - Mouvements circulaires / zig-zags (forte variabilité de COG)
      - Arrêts / quasi-arrêts prolongés (> 1h) hors zone portuaire
      - Transbordement (rendez-vous prolongé à basse vitesse avec un autre navire proche, typiquement pétrolier)
      - Rencontre à 12 milles marins des côtes
    Ces règles sont basées sur des heuristiques simples pour la surveillance navale.
    Fallback:
      - Si peu de points en mémoire, on tente de charger l'archive JSONL du jour/j-1
      - Si <5 points au total, on applique des heuristiques 'low-data'
    """
    # 1) Résoudre le navire depuis la logique existante
    mmsi, v, name = resolve_vessel_query(vessel_query, prefer_mmsi=last_selected_mmsi)
    if not v:
        return {"status": "not_found", "candidates": [{"mmsi": m, "name": n} for (m, n, _s) in vessel_candidates(vessel_query, k=5)]}

    key = str(mmsi)
    cutoff = time.time() - minutes*60

    # 2) Mémoire courte
    series = [p for p in ais_history.get(key, []) if p["ts"] >= cutoff]

    # 3) Fallback archive si trop peu de points
    if len(series) < 5:
        arch = _load_archive_points(key, since_ts=cutoff)
        if arch:
            # fusionner (et re-trier)
            series = sorted(series + arch, key=lambda r: r["ts"])

    # 4) Si toujours rien → message utile
    if len(series) == 0:
        return {"status": "insufficient_history", "mmsi": key, "name": name, "points": 0, "window_min": minutes}

    # 5) Calculs
    lats = np.array([p["lat"] for p in series], dtype=float)
    lons = np.array([p["lon"] for p in series], dtype=float)
    sogs = np.array([p.get("sog") or 0.0 for p in series], dtype=float)
    cogs = np.array([p.get("cog") or 0.0 for p in series], dtype=float)
    timestamps = np.array([p.get("ts") or 0.0 for p in series], dtype=float)


    def angular_std(deg_array):
        # écart-type circulaire (0..360°)
        rad = np.deg2rad(deg_array)
        C = np.mean(np.cos(rad))
        S = np.mean(np.sin(rad))
        R = np.sqrt(C*C + S*S)
        return np.rad2deg(np.sqrt(-2*np.log(max(R, 1e-9))))

    # métriques
    sog_med = float(np.median(sogs)) if sogs.size else None
    cog_std = float(angular_std(cogs)) if cogs.size > 1 else 0.0

    # “droiture” approximative de la trajectoire : distance borne/longueur
    def path_length():
        d = 0.0
        for i in range(1, len(series)):
            d += haversine_m(lats[i-1], lons[i-1], lats[i], lons[i])
        return d
    if len(series) >= 2:
        chord_m = haversine_m(lats[0], lons[0], lats[-1], lons[-1])
        length_m = path_length()
        straightness = chord_m / max(length_m, 1e-6)  # proche de 1 = droit, << 1 = sinueux
    else:
        chord_m = 0.0
        length_m = 0.0
        straightness = 1.0

    duration_hours = (timestamps[-1] - timestamps[0]) / 3600.0 if len(timestamps) > 1 else 0.0

    # 6) Règles d'anomalie
    flags = []
    suspicion = False
    reasons = []

    # Règle 1: Loitering (Vitesse basse < 5kn + Trajectoire non rectiligne)
    is_loitering = (
        sog_med is not None and sog_med < 5.0 and # < 5 nœuds
        duration_hours >= 0.5 and # Au moins 30 minutes d'historique
        (cog_std >= 40.0 or straightness <= 0.5) and # Forte variabilité de cap OU trajectoire peu droite
        # Vérifier si loin d'un port ou zone habituelle (heuristique simple: profondeur > 50m?)
        (emodnet_depth_sample(lats[-1], lons[-1]).get("avg") or 0) > 50 # Supposons >50m = pleine mer
    )
    if is_loitering:
        flags.append("loitering")
        suspicion = True
        reasons.append(f"Comportement de loitering: Vitesse basse ({sog_med:.1f} kn) et trajectoire erratique ({straightness:.2f} straightness, {cog_std:.1f}° COG std) sur {duration_hours:.1f}h en pleine mer.")

    # Règle 2: Arrêt prolongé (> 1h) hors zone portuaire
    is_stopped_prolonged = (
        sog_med is not None and sog_med < 1.0 and # < 1 nœud (quasi-arrêt)
        duration_hours >= 1.0 and # Au moins 1 heure
         # Vérifier si loin d'un port ou zone habituelle (heuristique simple: profondeur > 20m?)
        (emodnet_depth_sample(lats[-1], lons[-1]).get("avg") or 0) > 20 # Supposons >20m = hors zone portuaire facile
    )
    if is_stopped_prolonged:
        flags.append("stopped_prolonged")
        suspicion = True
        reasons.append(f"Arrêt prolongé ({sog_med:.1f} kn) sur {duration_hours:.1f}h en dehors d'une zone portuaire.")


    # Règle 3: Transbordement (STS - Ship-to-Ship)
    # Détecte si le navire analysé est à basse vitesse et a un ou plusieurs voisins très proches
    # qui sont également à basse vitesse, pour une durée prolongée.
    # Simplified logic to avoid persistent syntax error in detailed neighbor processing
    if sog_med is not None and sog_med < 3.0 and duration_hours >= 2.0: # Low speed prolonged (>= 2 hours)
        # Basic check for *any* slow neighbor nearby
        try:
            any_slow_neighbor_nearby = False
            for mm, vv in vessels_state.items():
                if str(mm) == key: continue
                la, lo = vv.get("lat"), vv.get("lon")
                if la is None or lo is None: continue
                try:
                    rng_m = haversine_m(lats[-1], lons[-1], la, lo) # Distance from last known point
                    if rng_m <= 0.5 * 1852.0: # 0.5 nm
                        neighbor_sog = vv.get("sog")
                        if neighbor_sog is not None and neighbor_sog < 3.0: # Neighbor is also slow
                             any_slow_neighbor_nearby = True
                             break # Found one, no need to check others
                except Exception:
                    continue # Skip if distance calculation fails for a neighbor

            if any_slow_neighbor_nearby:
                 flags.append("possible_sts")
                 suspicion = True
                 reasons.append(f"Basse vitesse prolongée ({sog_med:.1f} kn) sur {duration_hours:.1f}h avec au moins un voisin lent et très proche (<0.5nm). Suggère un possible transbordement (STS). Note: Analyse détaillée des voisins non disponible pour l'instant.")
                 is_sts_possible = True
        except Exception as e:
            print(f"Erreur détection STS simplifiée: {e}")
            reasons.append(f"Impossible de vérifier les voisins pour STS ({str(e)}).")


    # Règle 4: Rencontre près de 12 milles marins (Heuristique simple)
    # Détecte si le navire est à basse vitesse (< 5 kn) et à une distance de 10-15 NM de la côte la plus proche.
    # Ceci est une heuristique très simplifiée et nécessite des données précises sur la ligne de base côtière.
    is_near_12nm_boundary = False
    if sog_med is not None and sog_med < 5.0 and duration_hours >= 0.5:
        try:
            # Approximation grossière : distance à la côte la plus proche
            # Using the find_closest_coastal_country function's distance
            # Note: find_closest_coastal_country is not defined in this snippet,
            # assuming it exists globally or can be called.
            # For safety, let's add a check if it exists.
            if 'find_closest_coastal_country' in globals():
                closest_coast_info = find_closest_coastal_country(lats[-1], lons[-1])
                dist_nm = closest_coast_info['distance_nm']
                if 10.0 <= dist_nm <= 15.0: # Between 10 and 15 NM
                     flags.append("near_12nm_encounter")
                     suspicion = True
                     reasons.append(f"Vitesse basse ({sog_med:.1f} kn) près de la limite des 12 milles ({dist_nm:.1f} nm de la côte).")
                     is_near_12nm_boundary = True
            else:
                 print("Warning: find_closest_coastal_country not found. Skipping 12nm rule.")
        except Exception as e:
            print(f"Erreur détection 12nm: {e}")


    # Règle 5: Arrêt au-dessus d'un câble sous-marin (Nécessite données câbles - non implémenté ici)
    # is_above_cable = False
    # if is_stopped and depth_m is not None and depth_m < 200: # Stopped in relatively shallow water (where cables are)
    #     # Logic to check if current lat/lon is close to a known submarine cable route
    #     # Requires a spatial index of cable data, which is not available in this notebook
    #     pass
    # if is_above_cable:
    #     flags.append("stopped_above_cable")
    #     suspicion = True
    #     reasons.append("Arrêt prolongé au-dessus d'une zone de câble sous-marin.")


    # Low-data fallback (moins de points que nécessaire pour les règles complexes)
    low_data = len(series) < 5
    if low_data and sog_med is not None and sog_med < 3.0:
        flags.append("low_data_slow")
        if not suspicion: # Ne pas marquer comme suspect si déjà une règle plus forte l'a fait
            suspicion = True
            reasons.append(f"Vitesse très basse ({sog_med:.1f} kn) mais historique insuffisant ({len(series)} pts) pour une analyse complète.")


    # Finalisation du rapport
    report_status = "ok"
    if not suspicion and len(series) > 0:
        report_status = "no_anomalies"
        reasons.append("Aucune anomalie détectée selon les règles actuelles.")
    elif len(series) == 0:
         report_status = "insufficient_history"


    return {
        "status": report_status,
        "mmsi": key,
        "name": name,
        "window_min": minutes,
        "points": len(series),
        "duration_hours": round(duration_hours, 2),
        "sog_median_kn": round(sog_med, 2) if sog_med is not None else None,
        "cog_std_deg": round(cog_std, 1),
        "straightness_ratio": round(straightness, 2),
        "flags": flags,
        "low_data_mode": low_data,
        "suspect": suspicion,
        "reasons": reasons,
        # Removed sts_neighbors from the return dict to simplify,
        # the relevant info is included in the reasons
    }


tools = [ais_lookup, risk_assess,detect_anomalies]

print(f"✅ {len(tools)} tools LangChain définis: {[t.name for t in tools]}")

✅ 3 tools LangChain définis: ['ais_lookup', 'risk_assess', 'detect_anomalies']


# 🤖 Configuration de l'agent LangChain avec mémoire conversationnelle et fallback multi-modèles

In [491]:
import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.messages import HumanMessage, AIMessage

PRIMARY_MODEL   = "deepseek/deepseek-chat-v3.1:free"
FALLBACK_MODELS = ["openai/gpt-4o-mini"]  # fallback tool-calling

def _make_llm(model_name: str) -> ChatOpenAI:
    # OpenRouter est compatible OpenAI : on pointe ChatOpenAI vers OpenRouter
    return ChatOpenAI(
        model=model_name,
        temperature=0.2,
        api_key=OPENROUTER_API_KEY,
        base_url="https://openrouter.ai/api/v1",
        default_headers={
            "HTTP-Referer": "https://colab.research.google.com",
            "X-Title": "Boat Tracker Agent"
        },
    )

# 🧠 mémoire locale (liste de messages LangChain) + fenêtre glissante
chat_memory_lc = []          # HumanMessage / AIMessage
MEM_WINDOW_TURNS = 12        # on garde ~12 tours (≈ 24 messages)

def _prune_memory():
    excess = len(chat_memory_lc) - (2 * MEM_WINDOW_TURNS)
    if excess > 0:
        del chat_memory_lc[:excess]

def get_chat_history():
    return chat_memory_lc

def append_chat_history(user_text=None, ai_text=None):
    if user_text is not None:
        chat_memory_lc.append(HumanMessage(content=user_text))
    if ai_text is not None:
        chat_memory_lc.append(AIMessage(content=ai_text))
    _prune_memory()

# 👉 Référence implicite au "dernier navire utilisé" (mise à jour par les tools)
if "last_ref" not in globals():
    last_ref = {}  # ex: {"mmsi":"123456789","name":"MAYAR","lat":..., "lon":...}

# Adjusted system prompt to emphasize processing explicit vessel identifiers
system_txt = """Tu es un assistant naval opérationnel expert, spécialisé dans l'aide à la navigation maritime.

CONTEXTE OPÉRATIONNEL:
- Tu assistes des marins professionnels dans leurs opérations
- Tes réponses doivent être claires, précises et adaptées au vocabulaire maritime
- Utilise toujours les unités nautiques (nœuds, milles nautiques, etc.)

RÈGLES DE RÉFÉRENCE:
- **PRIORITÉ ABSOLUE:** Si la requête utilisateur contient un nom de navire ou un MMSI explicite, utilise CET identifiant pour toutes les analyses (lookup, risque, anomalies). Ignore la référence conversationnelle ou la sélection UI pour cette requête spécifique.
- Si l'utilisateur mentionne un nom de navire ou un MMSI, mets à jour ta référence de "navire actif" pour les requêtes futures (si la requête suivante ne contient pas d'identifiant explicite).
- Pour "ce navire", "il", "le navire" : utilise le navire actif (basé sur la dernière mention explicite ou la sélection UI si aucune mention explicite récente).
- En cas de doute sur l'identifiant du navire, demande une clarification plutôt que de deviner.

OUTILS OBLIGATOIRES:
- Pour toute question de position/localisation/identification : utilise ais_lookup
- Pour l'évaluation de risque : utilise risk_assess
- Pour la Détection d’anomalies : detect_anomalies

STYLE DE COMMUNICATION:
- Sois direct et factuel
- Structure tes réponses avec des puces claires
- Utilise des emojis pour la lisibilité (📍 🌊 ⚡ 🧭)
- Indique toujours le MMSI pour identification
- Signale les points d'attention avec ⚠️ ou 🚨

GESTION D'ERREURS:
- Si un outil échoue, explique clairement le problème
- Propose des alternatives quand possible
- Ne jamais inventer de données
"""

# 👇 Le prompt DOIT contenir 'agent_scratchpad' (MessagesPlaceholder) et on lui passe 'chat_history'
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_txt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

def _make_agent(model_name: str) -> AgentExecutor:
    llm = _make_llm(model_name)
    ag = create_tool_calling_agent(llm, tools, prompt)   # tool-calling agent moderne
    ex = AgentExecutor(agent=ag, tools=tools, verbose=False)
    ex._model_name = model_name
    return ex

agent = _make_agent(PRIMARY_MODEL)

def ask_naval_assistant(question: str):
    """Applique la mémoire + hint navire par défaut (UI/last_ref) et gère les fallbacks."""
    global agent
    history = get_chat_history()

    # The `resolve_vessel_query` function within the tools already handles
    # prioritizing the explicit query over the `prefer_mmsi` hint (which comes from last_selected_mmsi).
    # The issue seems to be the agent's reasoning before calling the tool.
    # The adjusted prompt above is intended to guide the agent's reasoning.
    # We don't need to pass last_selected_mmsi explicitly to invoke here,
    # as the tools themselves will get the hint via their parameter if the agent provides it.
    # The prompt guides the agent on *what* vessel query to formulate for the tool.

    candidates = [PRIMARY_MODEL] + FALLBACK_MODELS
    for model in candidates:
        if getattr(agent, "_model_name", None) != model:
            agent = _make_agent(model)
        for attempt in range(2):
            try:
                # Relying on the agent's ability to formulate the correct vessel_query
                # based on the explicit identifier in the user's input and the refined prompt.
                return agent.invoke({"input": question, "chat_history": history})

            except Exception as e:
                msg = str(e)
                # gestion simple du rate limit/provider
                if ("429" in msg) or ("rate limit" in msg.lower()) or ("temporarily rate-limited" in msg.lower()) or ("blocked for policy violation" in msg.lower()):
                    time.sleep(1.0 + attempt * 1.5)
                    continue
                raise # Renvoyer l'exception si ce n'est pas un problème de rate limit/policy

    return {"output": "⚠️ Tous les modèles sont momentanément indisponibles. Réessaie plus tard."}

print("✅ Agent LangChain configuré avec outils maritimes")

✅ Agent LangChain configuré avec outils maritimes


In [492]:
# 🔧 Créer un config.py complet pour HumanLLM

import os
os.chdir('/content/CollabToolBuilder')

# Configuration complète basée sur les besoins de HumanLLM
config_content = '''
# Configuration complète pour HumanLLM maritime
import os

# Cache configuration
PickleCacheActivated = False

# API Keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", "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")

# Database configuration (pas utilisé pour notre cas)
ELASTICSEARCH_URL = ""
CHROMADB_PATH = ""
vector_store_type = "none"

# Models configuration - Configuration minimale pour notre usage
MODELS_CONFIG_LIST = {
    "default_llm": {
        "model": "deepseek/deepseek-chat-v3.1:free",
        "api_key": "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",
        "base_url": "https://openrouter.ai/api/v1",
        "temperature": 0.2
    },
    "premium_llm": {
        "model": "deepseek/deepseek-chat-v3.1:free",
        "api_key": "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",
        "base_url": "https://openrouter.ai/api/v1",
        "temperature": 0.1
    }
}

# Interface configuration
INTERFACE_MODE = "notebook"
DEBUG_MODE = False

# Logging configuration
LOGGING_LEVEL = "INFO"

# WebSocket configuration (pas utilisé)
USE_WEBSOCKET = False

# Other required variables
DEFAULT_MODEL = "default_llm"
PREMIUM_MODEL = "premium_llm"

# Authentication (pas utilisé pour votre cas)
HTTP_BASIC_AUTH = None
'''

# Écrire le fichier complet
with open('config.py', 'w') as f:
    f.write(config_content)

print("✅ Fichier config.py complet créé")

# Vérifier le contenu
with open('config.py', 'r') as f:
    content = f.read()
    print(f"📄 Config créé avec {len(content)} caractères")
    if 'MODELS_CONFIG_LIST' in content:
        print("✅ MODELS_CONFIG_LIST présent dans le config")

✅ Fichier config.py complet créé
📄 Config créé avec 1399 caractères
✅ MODELS_CONFIG_LIST présent dans le config


In [493]:
# 🧠 HumanLLM avec popup interactive et mémorisation des préférences
import sys
sys.path.insert(0, '/content/CollabToolBuilder')

from humanllm import HumanLLM
from langchain_openai import ChatOpenAI
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import os

# Variables globales
humanllm_interaction_counter = 0
humanllm_learning_system = None
current_popup = None

# Système de préférences maritimes apprises
maritime_preferences = {
    "auto_include_weather": False,
    "auto_include_risk": False,
    "auto_include_traffic": False,
    "auto_include_bathymetry": False,
    "response_format": "standard",
    "favorite_vessels": [],
    "preferred_units": "nautical"
}

def save_preferences():
    """Sauvegarde les préférences apprises"""
    try:
        os.makedirs('/content/feedback', exist_ok=True)
        with open('/content/feedback/maritime_preferences.json', 'w') as f:
            json.dump(maritime_preferences, f, indent=2)
    except Exception as e:
        print(f"⚠️ Erreur sauvegarde préférences: {e}")

def load_preferences():
    """Charge les préférences sauvegardées"""
    global maritime_preferences
    try:
        with open('/content/feedback/maritime_preferences.json', 'r') as f:
            maritime_preferences.update(json.load(f))
        print("✅ Préférences maritimes chargées")
    except FileNotFoundError:
        print("ℹ️ Pas de préférences sauvegardées, utilisation des défauts")
    except Exception as e:
        print(f"⚠️ Erreur chargement préférences: {e}")

# Charger les préférences au démarrage
load_preferences()

def apply_learned_preferences(question, base_answer, vessel_name=None):
    """Applique les préférences apprises automatiquement"""
    enhanced_answer = base_answer

    if maritime_preferences["auto_include_weather"] and "position" in question.lower():
        enhanced_answer += "\n\n🌊 **[Météo automatique - HumanLLM]**\n⏳ Récupération conditions météo..."

    if maritime_preferences["auto_include_risk"] and any(word in question.lower() for word in ["navire", "position", "situe"]):
        enhanced_answer += "\n\n⚠️ **[Évaluation risque automatique - HumanLLM]**\n🔄 Analyse des risques en cours..."

    if maritime_preferences["auto_include_traffic"]:
        enhanced_answer += "\n\n🚢 **[Trafic local automatique - HumanLLM]**\n📡 Scan des navires à proximité..."

    if vessel_name and vessel_name not in maritime_preferences["favorite_vessels"]:
        maritime_preferences["favorite_vessels"].append(vessel_name)
        if len(maritime_preferences["favorite_vessels"]) > 10:
            maritime_preferences["favorite_vessels"] = maritime_preferences["favorite_vessels"][-10:]
        save_preferences()

    return enhanced_answer

def show_humanllm_popup(question, answer, interaction_num):
    """Affiche la vraie popup interactive HumanLLM"""
    global current_popup

    # Analyser la question pour des suggestions intelligentes
    question_lower = question.lower()
    suggestions = []

    if any(word in question_lower for word in ["où", "position", "situe"]):
        suggestions.extend([
            'Inclure automatiquement la météo pour les positions',
            'Ajouter l\'évaluation des risques par défaut',
            'Afficher le trafic maritime local'
        ])

    if any(word in question_lower for word in ["navire", "bateau"]):
        suggestions.append('Mémoriser ce navire comme favori')

    if not suggestions:
        suggestions = [
            'Réponses plus détaillées par défaut',
            'Format de coordonnées amélioré',
            'Inclure contexte maritime étendu'
        ]

    # Widgets de la popup
    popup_title = widgets.HTML(
        value=f"""<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                           padding: 15px; border-radius: 8px; margin-bottom: 15px;'>
                    <h3 style='color: white; margin: 0;'>🤖 HumanLLM - Intervention #{interaction_num}</h3>
                    <p style='color: #f0f0f0; margin: 5px 0 0 0;'>Système d'apprentissage maritime actif</p>
                  </div>"""
    )

    analysis_text = widgets.HTML(
        value=f"""<div style='padding: 10px; background: #f8f9fa; border-radius: 5px; margin-bottom: 15px;'>
                    <p><strong>📊 Analyse de votre question :</strong></p>
                    <p style='font-style: italic; color: #666;'>"{question}"</p>
                    <p><strong>🎯 Améliorations détectées possibles :</strong></p>
                  </div>"""
    )

    # Suggestions d'amélioration
    improvements = widgets.SelectMultiple(
        options=suggestions,
        description='Sélectionnez:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%', height='120px')
    )

    # Zone de commentaire libre
    custom_improvement = widgets.Textarea(
        placeholder="Ou décrivez votre propre amélioration souhaitée...",
        description='Suggestion libre:',
        layout=widgets.Layout(width='100%', height='60px'),
        style={'description_width': 'initial'}
    )

    # Boutons d'action
    apply_btn = widgets.Button(
        description='✅ Appliquer les améliorations',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )

    skip_btn = widgets.Button(
        description='⏭️ Ignorer cette fois',
        button_style='warning',
        layout=widgets.Layout(width='150px')
    )

    always_btn = widgets.Button(
        description='🔄 Toujours appliquer',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )

    # Status de l'action
    status_output = widgets.HTML(value="")

    # Container de la popup
    current_popup = widgets.VBox([
        popup_title,
        analysis_text,
        improvements,
        custom_improvement,
        widgets.HBox([apply_btn, skip_btn, always_btn],
                     layout=widgets.Layout(justify_content='space-around')),
        status_output
    ], layout=widgets.Layout(
        border='2px solid #667eea',
        border_radius='10px',
        padding='20px',
        background_color='white',
        box_shadow='0 4px 12px rgba(0,0,0,0.15)',
        margin='10px 0'
    ))

    # Handlers pour les boutons
    def on_apply(_):
        selected = list(improvements.value)
        custom = custom_improvement.value.strip()

        # Appliquer les améliorations sélectionnées
        for improvement in selected:
            if 'météo' in improvement.lower():
                maritime_preferences["auto_include_weather"] = True
            if 'risque' in improvement.lower():
                maritime_preferences["auto_include_risk"] = True
            if 'trafic' in improvement.lower():
                maritime_preferences["auto_include_traffic"] = True
            if 'favori' in improvement.lower():
                # Extraire le nom du navire de la question
                import re
                match = re.search(r"(?:navire|bateau)\s+([A-Za-z0-9\s\-_]+)", question, re.IGNORECASE)
                if match:
                    vessel_name = match.group(1).strip()
                    if vessel_name not in maritime_preferences["favorite_vessels"]:
                        maritime_preferences["favorite_vessels"].append(vessel_name)

        save_preferences()

        status_output.value = f"""<div style='color: #198754; padding: 10px; background: #d1e7dd; border-radius: 5px; margin-top: 10px;'>
                                    ✅ Améliorations appliquées: {len(selected)} sélections
                                    {f'<br>💬 Suggestion libre: {custom}' if custom else ''}
                                    <br>🧠 HumanLLM va maintenant personnaliser vos prochaines réponses !
                                  </div>"""

        print(f"✅ HumanLLM: {len(selected)} améliorations appliquées")
        if custom:
            print(f"💬 Suggestion utilisateur: {custom}")

    def on_skip(_):
        status_output.value = """<div style='color: #ffc107; padding: 10px; background: #fff3cd; border-radius: 5px; margin-top: 10px;'>
                                   ⏭️ Améliorations ignorées cette fois
                                   <br>ℹ️ HumanLLM continuera d'observer vos interactions
                                 </div>"""
        print("⏭️ HumanLLM: Améliorations ignorées")

    def on_always(_):
        # Appliquer TOUTES les suggestions par défaut
        maritime_preferences["auto_include_weather"] = True
        maritime_preferences["auto_include_risk"] = True
        maritime_preferences["auto_include_traffic"] = True
        save_preferences()

        status_output.value = """<div style='color: #0dcaf0; padding: 10px; background: #cff4fc; border-radius: 5px; margin-top: 10px;'>
                                   🔄 Mode automatique activé !
                                   <br>🧠 HumanLLM va maintenant enrichir automatiquement toutes vos réponses
                                   <br>⚙️ Vous pouvez modifier ces préférences à tout moment
                                 </div>"""
        print("🔄 HumanLLM: Mode automatique complet activé")

    # Connecter les handlers
    apply_btn.on_click(on_apply)
    skip_btn.on_click(on_skip)
    always_btn.on_click(on_always)

    # Afficher la popup
    display(current_popup)

    return current_popup

# Configuration HumanLLM (comme avant)
try:
    learning_llm = ChatOpenAI(
        model="deepseek/deepseek-chat-v3.1:free",
        temperature=0.2,
        api_key=OPENROUTER_API_KEY,
        base_url="https://openrouter.ai/api/v1",
        default_headers={
            "HTTP-Referer": "https://colab.research.google.com",
            "X-Title": "Learning HumanLLM"
        }
    )

    humanllm_learning_system = HumanLLM(
        system_prompt="Tu es un système d'apprentissage maritime qui observe les interactions utilisateur.",
        llmORchains_list={
            "default_llm": learning_llm,
            "premium_llm": learning_llm
        },
        default_llmORchain="default_llm",
        premium_llmORchain="premium_llm",
        premium_llm_by_default=False,
        skip_rounds=2  # Test avec 2
    )

    print("✅ HumanLLM avec popup interactive configuré (skip_rounds=2)")

except Exception as e:
    print(f"⚠️ HumanLLM non disponible: {e}")
    humanllm_learning_system = None

def check_humanllm_intervention(question, answer):
    """Vérifie si HumanLLM doit intervenir avec popup interactive"""
    global humanllm_interaction_counter

    humanllm_interaction_counter += 1

    print(f"🔢 [HumanLLM] Interaction #{humanllm_interaction_counter}")

    if humanllm_interaction_counter % 2 == 0:
        print(f"🚨 [HumanLLM] INTERVENTION #{humanllm_interaction_counter} DÉCLENCHÉE")

        # Afficher la vraie popup interactive
        popup = show_humanllm_popup(question, answer, humanllm_interaction_counter)

        return True, popup

    return False, None

print("✅ Système HumanLLM interactif complet configuré")


ℹ️ Pas de préférences sauvegardées, utilisation des défauts
✅ HumanLLM avec popup interactive configuré (skip_rounds=2)
✅ Système HumanLLM interactif complet configuré


# 💬 Handler chat avancé avec formatage maritime et gestion historique

In [494]:
import re
from html import escape

def _extract_candidate_name(q: str) -> str:
    # Look for "navire X", "bateau Y", or a standalone name/MMSI
    m = re.search(r"(?:navire|bateau)\s+([A-Za-z0-9\-_/\. ]{2,})", q, flags=re.I)
    if m:
        return m.group(1).strip()
    # Also look for standalone potential names or MMSIs (simple heuristic)
    # Avoid short words that are likely not names
    candidates = re.findall(r"\b([A-Za-z0-9\-_/\. ]+)\b", q)
    for cand in reversed(candidates): # Check from the end, often where the subject is
        if len(cand) > 2 and not any(word in cand.lower() for word in ["le", "la", "un", "une", "du", "des", "à", "au", "aux", "est", "il", "elle", "ce", "cet", "cette", "ces", "mon", "ma", "mes", "ton", "ta", "tes", "son", "sa", "ses", "notre", "nos", "votre", "vos", "leur", "leurs", "quel", "quelle", "quels", "quelles", "type", "pays", "comportement", "suspect", "anomalie", "risque", "transbordement", "proche", "distance", "situe", "ou", "analyse", "check", "trouve", "donne"]):
            return cand.strip()

    return q.strip() # Return original query if no specific candidate found

def _format_risk(r: dict) -> str:
    if r.get("status") != "ok":
        if r.get("status") == "not_found":
            cands = r.get("candidates") or []
            if cands:
                cand_str = ", ".join([f"{c['name']} (MMSI {c['mmsi']})" for c in cands])
                return "Navire introuvable dans le snapshot AIS. Vouliez-vous dire: " + cand_str + " ?"
            return "Navire introuvable dans le snapshot AIS pour l’instant."
        if r.get("status") == "no_coords":
            return "Coordonnées du navire indisponibles pour le moment."
        return "Données de risque indisponibles pour le moment."

    bft = r.get("beaufort")
    bft_txt = f"Bft {bft} ({bft_label(bft)})" if bft is not None else "ND"
    parts = []
    parts.append(f"Risque **{r['risk_level']}** (score {r['risk_score']}).")
    parts.append(f"Vent {bft_txt}")
    if r.get("swh_m") is not None: parts.append(f"Houle ~{r['swh_m']:.1f} m (Copernicus)")
    if r.get("depth_m_LAT") is not None: parts.append(f"Profondeur ~{r['depth_m_LAT']:.1f} m LAT")
    if r.get("ukc_m") is not None: parts.append(f"UKC ~{r['ukc_m']:.1f} m")
    parts.append(f"Trafic local: {r['neighbors']} bateau(x) ≤2 nm")
    neigh = r.get("neighbors_top5") or []
    if neigh:
        top = ", ".join([f"{n['name']} ({n['range_nm']} nm)" for n in neigh[:3]])
        parts.append(f"Plus proches: {top}")
    head = " ".join(parts)
    tail = f"Navire: {r['name']} (MMSI {r['mmsi']})."
    return head + "\n" + tail


def _format_anomalies_marine_friendly(a: dict) -> str:
    """Formate le rapport d'anomalies dans un langage adapté aux marins"""
    if a.get("status") != "ok" and a.get("status") != "no_anomalies":
         if a.get("status") == "not_found":
            cands = a.get("candidates") or []
            if cands:
                cand_str = ", ".join([f"{c['name']} (MMSI {c['mmsi']})" for c in cands[:3]])
                return f"❌ Navire '{a.get('name', 'inconnu')}' non localisé ou historique insuffisant. Navires similaires: {cand_str}"
            return f"❌ Navire '{a.get('name', 'inconnu')}' non détecté ou historique insuffisant."
         if a.get("status") == "insufficient_history":
             return f"ℹ️ Historique AIS insuffisant ({a.get('points', 0)} points sur {a.get('window_min', 60)} min) pour analyser le comportement de {a.get('name', f'MMSI {a.get("mmsi", "N/A")}')}."
         return "❌ Détection d'anomalies temporairement indisponible."


    # Nom du navire et MMSI
    ship_name = a.get('name', 'Navire inconnu')
    mmsi = a.get('mmsi', 'N/A')

    # Niveau de suspicion avec emoji
    suspect = a.get('suspect', False)
    suspicion_emoji = "🔴" if suspect else "🟢"

    # Construction du rapport
    report = [f"**{ship_name}** (MMSI {mmsi}) - Comportement {suspicion_emoji} **SUSPECT**" if suspect else f"**{ship_name}** (MMSI {mmsi}) - Comportement {suspicion_emoji} **NORMAL**"]
    report.append(f"• Période analysée: {a.get('window_min', 60)} minutes ({a.get('points', 0)} points AIS)")

    # Métriques clés (if available and relevant)
    if a.get('sog_median_kn') is not None:
         report.append(f"• **Vitesse médiane**: {a['sog_median_kn']:.1f} nœuds")
    if a.get('straightness_ratio') is not None: # Always show straightness
         report.append(f"• **Rectitude trajectoire**: {a['straightness_ratio']:.2f} (1.0 = Droit, 0.0 = Sinueux)")
    if a.get('cog_std_deg') is not None: # Always show COG std
         report.append(f"• **Variabilité cap**: {a['cog_std_deg']:.1f}° STD")

    # Reasons for suspicion (if suspect)
    reasons = a.get("reasons", [])
    if suspect and reasons:
        report.append("**🚨 Raisons de la suspicion:**")
        for reason in reasons:
            report.append(f"  - {reason}")
    elif not suspect and reasons:
         # Show status/reasons even if not suspect (e.g., "Aucune anomalie")
         for reason in reasons:
            report.append(f"• *Statut*: {reason}")


    # Detected flags
    flags = a.get("flags", [])
    if flags:
        flag_descriptions = {
            "loitering": "Comportement de loitering (vitesse basse, trajectoire non rectiligne)",
            "stopped_prolonged": "Arrêt/Quasi-arrêt prolongé hors zone portuaire",
            "possible_sts": "Possible transbordement (rencontre prolongée basse vitesse avec un autre navire proche)",
            "low_data_slow": "Vitesse basse mais données insuffisantes pour analyse complète",
            "near_12nm_encounter": "Vitesse basse près de la limite des 12 milles"
        }
        report.append("**🚩 Flags détectés:**")
        for flag in flags:
            report.append(f"  - {flag_descriptions.get(flag, flag)}")

    # Include STS neighbor details if available
    sts_neighbors = a.get("sts_neighbors")
    if "possible_sts" in flags and sts_neighbors:
         neighbor_info = ", ".join([f'{n["name"]} ({n["type"]}/{n["flag_country"]} @{n["range_nm"]}nm)' for n in sts_neighbors[:3]]) # Show up to 3 neighbors
         report.append(f"**🚢 Voisins STS potentiels:** {neighbor_info}")


    return "\n".join(report)


def _make_marine_friendly(text: str) -> str:
    """Adapte le langage technique pour les marins"""
    # Remplacements pour un langage plus maritime
    replacements = {
        "latitude": "latitude",
        "longitude": "longitude",
        "MMSI": "MMSI",
        "nœuds": "nœuds",
        "degrés": "°",
        "mètres": "m",
        "kilomètres": "km",
        "nautique": "nautique",
        "vitesse sur le fond": "vitesse",
        "cap sur le fond": "route",
        "à l'arrêt": "stoppé",
        "immobilisé": "au mouillage/stoppé"
    }

    result = text
    for old, new in replacements.items():
        result = result.replace(old, new)

    # Améliorer la structure des réponses
    if "Position:" in result:
        result = result.replace("Position:", "📍 **Position actuelle:**")
    if "Vitesse:" in result:
        result = result.replace("Vitesse:", "⚡ **Vitesse:**")
    if "Cap:" in result:
        result = result.replace("Cap:", "🧭 **Route:**")

    return result

def _render_history_html():
    if not chat_memory_lc:
        return """<div style='padding:15px;background:#f7f8fa;border-radius:8px;color:#2c3e50;text-align:center;font-style:italic;'>
                    🤖 Aucun échange pour l'instant. Posez votre première question !
                  </div>"""

    rows = []
    for i, msg in enumerate(chat_memory_lc):
        role = "Utilisateur" if isinstance(msg, type(HumanMessage(content=''))) else "Assistant"

        if role == "Utilisateur":
            cls = "background:linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%);border-left:4px solid #2196f3;"
            icon = "👤"
        else:
            cls = "background:linear-gradient(135deg, #f1f8e9 0%, #dcedc8 100%);border-left:4px solid #4caf50;"
            icon = "🤖"

        content = escape(msg.content).replace("\n", "<br>")
        timestamp = f"Message {i//2 + 1}" if role == "Utilisateur" else ""

        rows.append(f"""
        <div style='margin:8px 0;padding:12px 15px;border-radius:8px;{cls}box-shadow:0 2px 4px rgba(0,0,0,0.1);'>
            <div style='display:flex;align-items:center;margin-bottom:5px;'>
                <span style='margin-right:8px;font-size:16px;'>{icon}</span>
                <b style='color:#1976d2;'>{role}</b>
                {f'<span style="margin-left:auto;font-size:12px;color:#666;">{timestamp}</span>' if timestamp else ''}
            </div>
            <div style='color:#2c3e50;line-height:1.4;'>{content}</div>
        </div>
        """)

    return f"""<div style='max-height:350px;overflow-y:auto;padding:10px;background:#fafafa;border-radius:8px;'>
                 {"".join(rows)}
               </div>"""

def handle_chat(_button):
    global last_question, last_answer  # Variables pour le feedback

    question = chat_input.value.strip()
    if not question:
        return

    # Mémoriser la question pour le feedback
    last_question = question

    # 💾 ajoute la question dans la mémoire AVANT l'appel agent
    append_chat_history(user_text=question)

    chat_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Réflexion en cours...</div>"

    # ✅ NOUVEAU : Extraire l'identifiant du navire de la requête
    # Use the extracted identifier as the primary input for the agent
    vessel_identifier = _extract_candidate_name(question)
    print(f"🔎 Extracted potential vessel identifier: '{vessel_identifier}'")

    # ❌ REMOVED fast-paths here to ensure agent handles all specific maritime queries
    # The agent's prompt and tool descriptions should guide it to use the correct tools

    try:
        # Send the original question to the agent. The agent will use its tools
        # based on the refined prompt which emphasizes using explicit identifiers.
        # The tools themselves (like ais_lookup) also prioritize the query parameter.
        res = ask_naval_assistant(question)
        answer = res.get("output") or str(res)
    except Exception as e:
        answer = f"❌ Erreur de l'agent IA: {e}"


    # Mémoriser la réponse pour le feedback
    last_answer = answer

    # 💾 ajoute la réponse dans la mémoire APRÈS
    append_chat_history(ai_text=answer)

    # Rendu UI
    chat_output.value = f"""<div style='padding:10px;background:#fff8dc;border-radius:5px;color:#2c3e50;'>
    <h4 style='color:#34495e;margin-top:0;'>❓ Question: {escape(question)}</h4>
    <h4 style='color:#34495e;'>🤖 Assistant IA (outillé):</h4>
    <p style='white-space: pre-wrap; font-weight: 500; line-height: 1.5;'>{answer}</p>
    </div>"""

    # Afficher le panneau de feedback après chaque réponse
    feedback_status.value = ""
    feedback_thumbs.value = None
    feedback_reason.value = ''
    feedback_comment.value = ''
    feedback_panel.layout.display = ''  # Rendre visible

    history_output.value = _render_history_html()
    chat_input.value = ""

    # 🧠 Vérification intervention HumanLLM avec popup interactive
    if 'check_humanllm_intervention' in globals():
        try:
            # Vérifier intervention (avec la nouvelle signature)
            intervention_triggered, popup = check_humanllm_intervention(question, answer)

            if intervention_triggered:
                print("🎉 Popup HumanLLM affichée - Interaction avec l'utilisateur en cours...")

        except Exception as e:
            print(f"⚠️ Erreur HumanLLM: {e}")

# (Re)lier sans doublons
try:
    chat_button._click_handlers.callbacks = []
except Exception:
    pass
chat_button.on_click(handle_chat)

def handle_enter(_):
    if chat_input.value.strip():
        handle_chat(None)
chat_input.on_submit(handle_enter)

print("✅ Handler chat mis à jour avec HumanLLM interactif")

def handle_enter(_):
    if chat_input.value.strip():
        handle_chat(None)
chat_input.on_submit(handle_enter)

# Bouton "effacer l'historique"
def _clear_history(_btn):
    chat_memory_lc.clear()
    history_output.value = _render_history_html()
try:
    clear_history_button._click_handlers.callbacks = []
except Exception:
    pass
clear_history_button.on_click(_clear_history)

# Affiche l'historique courant au chargement
history_output.value = _render_history_html()

print("✅ Chat branché (mémoire multi-tours + référence implicite de navire).")

✅ Handler chat mis à jour avec HumanLLM interactif
✅ Chat branché (mémoire multi-tours + référence implicite de navire).


In [495]:
def history_debug(mmsi: str, minutes=60):
    key = str(mmsi)
    series = list(ais_history.get(key, []))
    print(f"🔎 {key} → {len(series)} points en mémoire courte")
    if series:
        print("  Dernier point:", series[-1])
    cutoff = time.time() - minutes*60
    recent = [p for p in series if p["ts"] >= cutoff]
    print(f"  → {len(recent)} points dans les {minutes} dernières minutes")

history_debug("271051231", minutes=60)

🔎 271051231 → 0 points en mémoire courte
  → 0 points dans les 60 dernières minutes


In [496]:
# 🧑‍⚖️ Système de feedback Human-in-the-Loop (HITL)
import os, json, datetime as dt
from html import escape

# Variables globales pour mémoriser la dernière Q/R
last_question, last_answer = None, None

# Widgets de feedback
feedback_thumbs = widgets.ToggleButtons(
    options=[('👍 Utile', 'up'), ('👎 Pas utile', 'down')],
    tooltips=['Réponse utile', 'Réponse insuffisante'],
    description='Avis:',
    style={'description_width': 'initial'}
)

feedback_reason = widgets.Dropdown(
    options=[('— Sélectionnez un motif —', ''),
             ('Erreur factuelle / halluciné', 'fact'),
             ("N'a pas utilisé les tools", 'tools'),
             ('Mauvaise identification navire', 'vessel'),
             ('Réponse floue/trop longue', 'clarity'),
             ('Manque de contexte maritime', 'context'),
             ('Problème UI/latence', 'ui'),
             ('Autre', 'other')],
    description='Raison:',
    style={'description_width': 'initial'}
)

feedback_comment = widgets.Textarea(
    placeholder="Expliquez brièvement ce qui allait/n'allait pas, ou donnez la bonne réponse...",
    layout=widgets.Layout(width='100%', height='80px')
)

feedback_send_btn = widgets.Button(
    description='📥 Enregistrer le feedback',
    icon='paper-plane',
    button_style='success'
)

feedback_retry_btn = widgets.Button(
    description="🔁 Relancer avec ma correction",
    icon='refresh',
    button_style='info'
)

feedback_status = widgets.HTML(value="")

# Affichage/masquage dynamique des champs détaillés quand 👎
def _on_feedback_change(change):
    if change['type'] == 'change' and change['name'] == 'value':  # ✅ LIGNE MANQUANTE AJOUTÉE
        need_detail = (change['new'] == 'down')
        feedback_reason.layout.display = '' if need_detail else 'none'
        feedback_comment.layout.display = '' if need_detail else 'none'

feedback_thumbs.observe(_on_feedback_change, names='value')

# État initial (caché tant qu'il n'y a pas de réponse)
feedback_reason.layout.display = 'none'
feedback_comment.layout.display = 'none'

# Conteneur du panneau feedback
feedback_panel = widgets.VBox([
    widgets.HTML(
        value="""<div style='background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%);
                           padding: 15px; border-radius: 8px; border-left: 4px solid #6c757d; margin: 10px 0;'>
                   <h3 style='margin: 0 0 5px 0; color: #495057;'>🧑‍⚖️ Feedback sur la réponse</h3>
                   <p style='margin: 0; color: #6c757d; font-size: 14px;'>Aidez-nous à améliorer l'assistant maritime</p>
                 </div>"""
    ),
    feedback_thumbs,
    feedback_reason,
    feedback_comment,
    widgets.HBox([feedback_send_btn, feedback_retry_btn],
                 layout=widgets.Layout(justify_content='flex-start')),
    feedback_status
], layout=widgets.Layout(display='none', margin='10px 0'))  # masqué au départ

print("✅ Widgets de feedback créés")

✅ Widgets de feedback créés


In [497]:
# 🗂️ Fonctions de gestion du feedback JSONL

def _ensure_feedback_dir():
    """Crée le dossier de feedback s'il n'existe pas"""
    os.makedirs('/content/feedback', exist_ok=True)

def _save_feedback(payload: dict) -> str:
    """Enregistre le feedback en JSONL"""
    _ensure_feedback_dir()
    path = '/content/feedback/ratings.jsonl'
    with open(path, 'a', encoding='utf-8') as f:
        f.write(json.dumps(payload, ensure_ascii=False) + '\n')
    return path

def _on_feedback_send(_btn):
    """Handler pour l'envoi du feedback"""
    if not (last_question and last_answer):
        feedback_status.value = "<div style='color:#dc3545;padding:8px;'>⚠️ Pas de réponse à évaluer.</div>"
        return

    # Déterminer si actionnable (pour amélioration future)
    is_actionable = (
        feedback_thumbs.value == 'down' and
        feedback_comment.value.strip() and
        len(feedback_comment.value.strip()) > 10
    )

    payload = {
        "ts": dt.datetime.utcnow().isoformat() + "Z",
        "question": last_question,
        "answer": last_answer,
        "verdict": feedback_thumbs.value,  # 'up' | 'down'
        "reason": feedback_reason.value,
        "comment": feedback_comment.value.strip(),
        "selected_mmsi": last_selected_mmsi,
        "actionable": is_actionable,
        "model": "deepseek-chat-v3.1"  # ou récupérer dynamiquement
    }

    try:
        path = _save_feedback(payload)
        feedback_status.value = f"""<div style='color:#198754;padding:8px;background:#d1e7dd;border-radius:4px;'>
                                      ✅ Merci ! Feedback enregistré dans <b>{os.path.basename(path)}</b>
                                    </div>"""
        # Reset léger
        feedback_thumbs.value = None
        feedback_reason.value = ''
        feedback_comment.value = ''

    except Exception as e:
        feedback_status.value = f"<div style='color:#dc3545;padding:8px;'>❌ Erreur sauvegarde: {str(e)}</div>"

def _on_feedback_retry(_btn):
    """Handler pour relancer avec correction utilisateur"""
    base_q = (last_question or "").strip()
    critique = (feedback_comment.value or "").strip()

    if not base_q:
        feedback_status.value = "<div style='color:#dc3545;padding:8px;'>⚠️ Aucune question précédente.</div>"
        return

    if not critique:
        feedback_status.value = "<div style='color:#ffc107;padding:8px;'>⚠️ Ajoutez un commentaire pour guider la correction.</div>"
        return

    # Construire la requête avec correction
    enhanced_question = f"""{base_q}

[Retour utilisateur] {critique}
Corrige les points signalés et donne une réponse améliorée, en restant concis et en utilisant les outils AIS/météo si nécessaire."""

    # Remplir le champ de chat et masquer le panneau
    chat_input.value = enhanced_question
    feedback_panel.layout.display = 'none'

    # Relancer automatiquement
    handle_chat(None)

# Connecter les handlers
feedback_send_btn.on_click(_on_feedback_send)
feedback_retry_btn.on_click(_on_feedback_retry)

print("✅ Handlers de feedback configurés")

✅ Handlers de feedback configurés


# 🚀 Interface principale complète avec historique et instructions d'utilisation


In [498]:
# Affichage initial de la carte

with map_output:
    display(create_boat_map(vessels_state_to_df()))

# Création de l'interface avec un design moderne incluant l'historique
header = widgets.HTML(
    value="""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                padding: 20px; border-radius: 10px; margin-bottom: 20px;'>
        <h1 style='color: white; text-align: center; margin: 0; font-family: Arial, sans-serif;'>
            🚢 Système de Tracking Maritime
        </h1>
        <p style='color: #f0f0f0; text-align: center; margin: 10px 0 0 0;'>
            Suivez vos bateaux en temps réel avec météo, géolocalisation et assistance IA
        </p>
    </div>
    """
)


print("\n🎉 Application de tracking maritime lancée avec success!")
print("\n📝 Instructions d'utilisation:")
print("1. 🔑 Remplacez les clés API par vos vraies clés dans la section 'Configuration des clés API'")
print("2. 🎯 Sélectionnez un bateau dans le menu déroulant")
print("3. 🗺️ Visualisez sa position sur la carte (point rouge)")
print("4. 🌤️ Consultez les informations météo dans la sidebar")
print("5. 🌍 Vérifiez la géolocalisation (pays, ville, zone maritime)")
print("6. 💬 Posez des questions à l'assistant IA sur les bateaux")
print("7. 📜 Consultez l'historique de vos discussions")
print("8. 🗑️ Effacez l'historique quand necessaire")
print("\n✨ Fonctionnalités:")
print("• 🚢 Plusieurs bateaux reels avec clé API AIstream")
print("• 🌤️ Données météo en temps reel (OpenWeather)")
print("• 🌍 Géolocalisation precise (pays, ville, zone maritime)")
print("• 🤖 Chatbot IA intégré (Langchain + OpenRouter)")
print("• 📜 Historique de discussion persistant")
print("• 🎨 Interface moderne et responsive")
print("• 🗺️ Carte interactive avec marqueurs colorés + bathymetrie")

# Layout principal avec historique
main_layout = widgets.VBox([
    header,

    # Sélecteur de bateau
    widgets.HTML("<h2>🎯 Sélection du bateau</h2>"),
    boat_selector,


    # Layout horizontal pour la carte et la sidebar
    widgets.HBox([
        # Carte (côté gauche)
        widgets.VBox([
            widgets.HTML("<h2>🗺️ Carte de tracking</h2>"),
            map_output
        ], layout=widgets.Layout(width='60%')),

        # Sidebar (côté droit)
        widgets.VBox([
            widgets.HTML("<h2>📊 Informations</h2>"),
            weather_output,
            widgets.HTML("<br>"),
            location_output
        ], layout=widgets.Layout(width='40%', padding='0 0 0 20px'))
    ]),

    # Section chatbot avec historique
    widgets.HTML("<br><h2>🤖 Assistant IA Maritime</h2>"),

    # Zone de chat actuelle avec feedback
widgets.VBox([
    widgets.HTML("<h3>💬 Nouvelle question</h3>"),
    widgets.HBox([chat_input, chat_button], layout=widgets.Layout(width='100%')),
    chat_output,
    feedback_panel  # 👈 AJOUT DU PANNEAU DE FEEDBACK
], layout=widgets.Layout(margin='0 0 20px 0')),

    # Zone d'historique
    widgets.VBox([
        widgets.HBox([
            widgets.HTML("<h3>📜 Historique des discussions</h3>"),
            clear_history_button
        ], layout=widgets.Layout(justify_content='space-between', align_items='center')),
        widgets.HTML(
            value="<div style='max-height: 400px; overflow-y: auto; border: 1px solid #ddd; border-radius: 5px;'></div>",
            layout=widgets.Layout(width='100%')
        ),
        history_output
    ], layout=widgets.Layout(
        border='1px solid #e0e0e0',
        border_radius='8px',
        padding='15px',
        background_color='#fafafa'
    ))

], layout=widgets.Layout(padding='20px'))

boat_selector.observe(on_boat_selection_change, names='value')
print("✅ Handler de sélection connecté au dropdown")

# Affichage de l'application
display(main_layout)


🎉 Application de tracking maritime lancée avec success!

📝 Instructions d'utilisation:
1. 🔑 Remplacez les clés API par vos vraies clés dans la section 'Configuration des clés API'
2. 🎯 Sélectionnez un bateau dans le menu déroulant
3. 🗺️ Visualisez sa position sur la carte (point rouge)
4. 🌤️ Consultez les informations météo dans la sidebar
5. 🌍 Vérifiez la géolocalisation (pays, ville, zone maritime)
6. 💬 Posez des questions à l'assistant IA sur les bateaux
7. 📜 Consultez l'historique de vos discussions
8. 🗑️ Effacez l'historique quand necessaire

✨ Fonctionnalités:
• 🚢 Plusieurs bateaux reels avec clé API AIstream
• 🌤️ Données météo en temps reel (OpenWeather)
• 🌍 Géolocalisation precise (pays, ville, zone maritime)
• 🤖 Chatbot IA intégré (Langchain + OpenRouter)
• 📜 Historique de discussion persistant
• 🎨 Interface moderne et responsive
• 🗺️ Carte interactive avec marqueurs colorés + bathymetrie
✅ Handler de sélection connecté au dropdown


📊 200000 messages AIS reçus, 1823 navires en mémoire
📊 154500 messages AIS reçus, 1833 navires en mémoire
🔎 Extracted potential vessel identifier: 'barra'
🔢 [HumanLLM] Interaction #1
📊 201600 messages AIS reçus, 2206 navires en mémoire
📊 155700 messages AIS reçus, 2208 navires en mémoire
🔎 Extracted potential vessel identifier: 'analyse son comportement ?'
🔢 [HumanLLM] Interaction #2
🚨 [HumanLLM] INTERVENTION #2 DÉCLENCHÉE


🎉 Popup HumanLLM affichée - Interaction avec l'utilisateur en cours...


# ⏰ Premier refresh interface + refresh manuel (à relancer pour actualiser)

In [500]:
# Attendre que le flux AIS commence à recevoir des données
time.sleep(5)

# Premier refresh pour afficher les navires détectés
refresh_ui_from_stream()

print("✅ Premier refresh effectué")
print("📝 IMPORTANT: Pour actualiser la liste des navires, relancez cette cellule")
print("   ou utilisez la boucle automatique ci-dessous")

# OU lancer une boucle auto (toutes les 10s, 30 fois) :
# run_refresh_loop(seconds=10, iterations=30)

print("\n🔄 Options de refresh :")
print("• Relancer cette cellule manuellement quand vous voulez actualiser")
print("• Décommenter la ligne run_refresh_loop() pour un refresh automatique")
print("• Les nouveaux navires AIS apparaîtront dans le dropdown après refresh")


✅ Premier refresh effectué
📝 IMPORTANT: Pour actualiser la liste des navires, relancez cette cellule
   ou utilisez la boucle automatique ci-dessous

🔄 Options de refresh :
• Relancer cette cellule manuellement quand vous voulez actualiser
• Décommenter la ligne run_refresh_loop() pour un refresh automatique
• Les nouveaux navires AIS apparaîtront dans le dropdown après refresh
